![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Coco Lab

# Setting Up A Clinical Trial

## Introduction

Clinical trials are used to test that new treatments are both safe and effective.  They involve taking measurements from various patients both before and after they start the treatment.

The data comes in from a variety of hospitals.  It is personally sensitive to the patients, of importance to the business, and subject to regulatory control and so care is needed that:

* it has been collected correctly
* it is protected at all times
* the correct data sharing agreements are in place to provide legal cover, both for the hospitals and Coco Pharmaceuticals.

We look at this process from the perspective of

1. The **business (clinical research)** who manage the clinical trial.  Tessa Tube is Coco Pharmaceutical's Chief Research Officer. She commissions any new trial, sets out the requirements and objectives for the study.  Tanya Tidie is responsible for the administration of the clinical trial.  She organizaes the data sharing agreements with the hospitals and monitors the progress to ensure all of the evidence is in place to show to the regulators.
   
2. The **data engineer** who sets up the data pipelines. Peter Profile is responsible for selecting the Unity Catalog that will protect the data from the clinical trial, setting up a schema and volume for the weekly measurements data in that catalog and, as the hospitals complete their registration into the clinical trail, activate the landing area and onboarding pipeline for data fro mthe hospital.

3. The **data scientist** who analyses the resulting data.  Callie Quartile is the data scientist who has to locate, access and analyse the data.  She needs to be sure the data is of sufficient quality and she has the right permissions to perform the type of analysis she wants.
   
<figure style="margin-left: 0%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/tessa-tube.png">
<figcaption style="margin-left: 15%;"><strong>Tessa Tube</strong></figcaption>
</figure>
    
<figure style="margin-left: 7%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/tanya-tidie.png">
<figcaption style="margin-left: 15%;"><strong>Tanya Tidie</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/peter-profile.png">
<figcaption style="margin-left: 15%;"><strong>Peter Profile</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">
<img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/callie-quartile.png">
<figcaption style="margin-left: 15%;"><strong>Callie Quartile</strong></figcaption>
</figure>


----

Lets get started by initializing the python interface to Egeria - pyegeria.

----

In [ ]:
# Initialize pyegeria

%run ../pyegeria/initialize-pyegeria.ipynb

from datetime import datetime


----

The code below creates a client to validate the set up.

----

In [ ]:
user_id = "erinoverview"
egeria_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = egeria_client.create_egeria_bearer_token()

----

This content pack contains the definitions of the Coco Pharmaceuticals organization.

----

In [ ]:

egeria_client.add_archive_file("content-packs/CocoComboArchive.omarchive", None, "qs-metadata-store")

print("CocoComboArchive loaded!")


----

This checks that the governance engines that support the clinical trial are running ...

----

In [ ]:
display_gov_eng_status(['AssetOnboarding','AssetQuality@CocoPharmaceuticals','ClinicalTrials@CocoPharmaceuticals','UnityCatalogGovernance','UnityCatalogSurvey'],
                       status_filter=["*"],
                       engine_host = 'qs-engine-host',  view_server = 'qs-view-server',
                       paging = True, jupyter = True,width = 170,sort = True)

----

## Clinical Trial Solution Blueprint




In [ ]:

print_solution_blueprints(egeria_client, "Clinical Trial Management Solution Blueprint")


----

## Initializing the clinical trial

Once Tessa has a clear idea of the scope and purpose of the clinical trial, she is ready to initiate it in Egeria.  Clinical trials are operated as projects that are part of the *Clinical Trial Management* campaign.  This is a project that holds the generic resources used by all clinical trials.

---

In [ ]:
user_id = "tessatube"
tessas_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = tessas_client.create_egeria_bearer_token()

In [ ]:

parent_project_name="Campaign:Clinical Trials Management"
parent_project_guid=tessas_client.get_element_guid_by_unique_name(parent_project_name)

print("Parent project: " + parent_project_name)
print("  parent project GUID: " + parent_project_guid)

print_project(tessas_client, parent_project_guid)


---

Tessa's role is the *Clinical Trial Sponsor*. Although she is monitor the clinical trial throughout its lifetime, she has two specific actions to perform, one at the start of the trial and one at the end:

---

In [ ]:

print_solution_roles(tessas_client, "Clinical Trial Sponsor")


---

Her first responsibility is to set up the clinical trial.

---

In [ ]:

print_solution_components(tessas_client, "Set up clinical trial")


---

The `set-up-clinical-trial` governance process creates the specific governance processes that will be used by the staff during the clinical trial.  It uses generic process definitions to create these specific processes that are initialized with all of the correct values for the clinical trial.  This approach reduces the chance that someone will use a wrong value by accident when running the later processes for the clinical trial.

---

In [ ]:

set_up_clinical_trial_name="ClinicalTrials@CocoPharmaceuticals::set-up-clinical-trial"


----

The work of the clinical trial is managed under the `Teddy Bear Drop Foot Clinical Trial` project.

----

In [ ]:
print()

project_identifier="PROJ-CT-TBDF"
project_name="Teddy Bear Drop Foot Clinical Trial"
project_description="Clinical trial related to the new treatment for Teddy Bear Drop Foot."
project_directory_name="teddy-bear-drop-foot"
project_schema_name="teddy_bear_drop_foot"

print("Project " + project_identifier + ": " + project_name)
print("  " + project_description)


----

The people involved in the clinical trial are identified.

----

In [ ]:
print()

clinical_trial_owner_name="Tessa Tube"
print("clinical trial owner name: " + clinical_trial_owner_name)
clinical_trial_owner_guid = tessas_client.get_guid_for_name(clinical_trial_owner_name)
print("clinical trial owner GUID: " + clinical_trial_owner_guid)

print()
clinical_trial_manager_name="Tanya Tidie"
print("clinical trial manager name: " + clinical_trial_manager_name)
clinical_trial_manager_guid = tessas_client.get_guid_for_name(clinical_trial_manager_name)
print("clinical trial manager GUID: " + clinical_trial_manager_guid)

print()
data_scientist_name="Callie Quartile"
print("data scientist name: " + data_scientist_name)
data_scientist_guid = tessas_client.get_guid_for_name(data_scientist_name)
print("data scientist GUID: " + data_scientist_guid)

print()
data_engineer_name="Peter Profile"
print("data engineer name: " + data_engineer_name)
data_engineer_guid = tessas_client.get_guid_for_name(data_engineer_name) 
print("data engineer GUID: " + data_engineer_guid)

print()
project_manager_name="Polly Tasker"
print("project manager name: " + project_manager_name)
project_manager_guid = tessas_client.get_guid_for_name(project_manager_name) 
print("project manager GUID: " + project_manager_guid)

print()
integration_developer_name="Bob Nitter"
print("integration developer name: " + integration_developer_name)
integration_developer_guid = tessas_client.get_guid_for_name(integration_developer_name) 
print("integration developer GUID: " + integration_developer_guid)

print()

----

All of the information is assembled to set up the clinical trial.

----

In [ ]:


action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "clinicalTrialParentProject",
      "actionTargetGUID": parent_project_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "clinicalTrialOwner",
      "actionTargetGUID": clinical_trial_owner_guid 
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "clinicalTrialManager",
      "actionTargetGUID": clinical_trial_manager_guid 
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "itProjectManager",
      "actionTargetGUID": project_manager_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "dataEngineer",
      "actionTargetGUID": data_engineer_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "integrationDeveloper",
      "actionTargetGUID": integration_developer_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "dataScientist",
      "actionTargetGUID": data_scientist_guid
    }]

request_parameters = request_parameters = {
     "clinicalTrialId" : project_identifier,
     "clinicalTrialName" : project_name,
     "clinicalTrialDescription" : project_description
}

tessas_client.initiate_gov_action_type(set_up_clinical_trial_name, None, action_targets, None, request_parameters, None, None)

In [ ]:

display_engine_activity_c(row_limit=1,width=170)


-----

The process has created the following processes that are specific to this clinical trial and are set up with details of the correct project and people.

-----

In [ ]:

print_matching_gov_action_processes(tessas_client, project_identifier)


---

These processes are linked to the appropriate tasks that were also defined by the set up process...

---

In [ ]:


project_guid=tessas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::" + project_name)
print_project_graph(tessas_client, project_guid)


----

## Nominating Hospitals

Tanya Tidie is the clinical trial manager, responsible for the business arrangements for each clinical trial. 

In [ ]:
user_id = "tanyatidie"
tanyas_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = tanyas_client.create_egeria_bearer_token()

In [ ]:

print_solution_roles(tanyas_client, "Clinical Trial Manager")


---

This includes managing the relationships with the hospitals, ensuring all of the "paperwork" associated with the clinical trial is in place for the sponosrs and regulators as well as handling any issues with the data received from the hospitals.

----

In [ ]:

project_guid=tanyas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::HospitalManagement")
print_project_graph(tanyas_client, project_guid)


----

Once the basic data sharing agreement has been defined, Tanya begins to work with the hospitals that will take part in the clinical trial.  For each one, she runs nominate-hospital to identify the people at the hospital that are going to complete the requirements (training, signing data sharing agreements, ...) necessary to participate.

----

In [ ]:

print_solution_components(tanyas_client, "Nominate Hospital")


---

The *Nominate Hospital* Governance Action Type is the generic function that is used to nominate a particular hospital.  For each clinical trial, it is wrapped in a new Governance Action Process that is configured with key information about the clinical trial.  This reduces the information needed to be supplied when nominating a hospital.

---

In [ ]:

nominate_hospital_name = "ClinicalTrials::" + project_identifier + "::nominate-hospital"

process_guid = tanyas_client.get_element_guid_by_unique_name(nominate_hospital_name)
#print_related_metadata_elements(tanyas_client, process_guid)

process_graph = tanyas_client.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)



----

### Nominating Oak Dene Hospital

----

In [ ]:
print()

oak_dene_hospital_name="Oak Dene Hospital"
print(oak_dene_hospital_name)
oak_dene_hospital_guid=tanyas_client.get_element_guid_by_unique_name("Organization::" + oak_dene_hospital_name)
print("  oakDeneHospitalGUID: " + oak_dene_hospital_guid)
oak_dene_contact_person_name="Robbie Records"
print("  oakDeneContactPersonName: " + oak_dene_contact_person_name)
oak_dene_contact_person_guid=tanyas_client.get_guid_for_name(oak_dene_contact_person_name)
print("  oakDeneContactPersonGUID: " + oak_dene_contact_person_guid)


In [ ]:

action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": oak_dene_hospital_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "hospitalContactPerson",
      "actionTargetGUID": oak_dene_contact_person_guid
    }]


tanyas_client.initiate_gov_action_process(nominate_hospital_name, None, action_targets, datetime.now(), None, None, None)

----

### Nominating Old Market Hospital

----

In [ ]:
print()
old_market_hospital_name="Old Market Hospital"
print(old_market_hospital_name)
old_market_hospital_guid=tanyas_client.get_element_guid_by_unique_name("Organization::" + old_market_hospital_name)
print("  oldMarketHospitalGUID: " + old_market_hospital_guid)
old_market_contact_person_name="Nellie Dunn"
print("  oldMarketContactPersonName: " + old_market_contact_person_name)
old_market_contact_person_guid=tanyas_client.get_guid_for_name(old_market_contact_person_name)
print("  oldMarketContactPersonGUID: " + old_market_contact_person_guid)
print()

In [ ]:

action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": old_market_hospital_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "hospitalContactPerson",
      "actionTargetGUID": old_market_contact_person_guid
    }]

tanyas_client.initiate_gov_action_process(nominate_hospital_name, None, action_targets, datetime.now(), None, None, None)

----

### Nominating Hampton Hospital

----

In [ ]:
print()
hampton_hospital_name="Hampton Hospital"
print(hampton_hospital_name)
hampton_hospital_guid=tanyas_client.get_element_guid_by_unique_name("Organization::" + hampton_hospital_name)
print("  hamptonHospitalGUID: " + hampton_hospital_guid)
hampton_contact_person_name="Grant Able"
print("  hamptonContactPersonName: " + hampton_contact_person_name)
hampton_contact_person_guid=tanyas_client.get_guid_for_name(hampton_contact_person_name)
print("  hamptonContactPersonGUID: " + hampton_contact_person_guid)

In [ ]:
action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": hampton_hospital_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "hospitalContactPerson",
      "actionTargetGUID": hampton_contact_person_guid
    }]


tanyas_client.initiate_gov_action_process(nominate_hospital_name, None, action_targets, datetime.now(), None, None, None)

In [ ]:
display_engine_activity_c(row_limit=3, width=150)

----

## Setting up the data lake

Peter Profile is a data engineer at Coco Pharmaceuticals and is reponsible for the data pipelines and data stores that support the clinical trial.

---

In [ ]:
user_id = "peterprofile"
peters_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = peters_client.create_egeria_bearer_token()

In [ ]:

print_solution_roles(peters_client, "Certified Data Engineer")


-----

Peter first task is to set up the data lake and the data stores that are independent of the hospital supplying the data.  This includes components and templates that are influenced by the shape of the data needed by Coco Pharmaceuticals to validate their new treatment.  These are 

---

In [ ]:

project_guid=tanyas_client.get_element_guid_by_unique_name("Project::" + project_identifier + "::OnboardingPipelines")
print_project_graph(peters_client, project_guid)


----

The process to be used for the data onboarding pipeline is `Onboard Landing Area Files For Teddy Bear Drop Foot Project`.

----

In [ ]:

onboarding_pipeline_name = "Onboard Landing Area Files For Clinical Trial Project"
onboarding_pipeline_guid = peters_client.get_guid_for_name(onboarding_pipeline_name) 

process_graph = peters_client.get_gov_action_process_graph(onboarding_pipeline_guid)
print_governance_action_process_graph(process_graph)


----

Data is received into the Coco Data Lake.  This is made up of multiple deployments of Unity Catalog, at the different sites with a variety of storage technologies attached.  Peter needs to select which Unity Catalog to use and then set it up for the specific clnical trial.

A Unity Catalog Server can contain multiple catalogs.  The code below assumes that you have a Unity Catalog Server running that is catalogued as `Unity Catalog 1` and there is a catalog within it called `clinical_trials`.  The **Unity Catalog Survey and Catalog Workbook** will set this up.  You can change the names if you like.

----

In [ ]:

#uc_server_url="http://host.docker.internal:8087"
uc_server_url="http://localhost:8087"
catalog_name="clinical_trials"

catalog_guid=peters_client.get_element_guid_by_unique_name("Unity Catalog Catalog::" + uc_server_url + "::" + catalog_name)
print(catalog_guid)


----

[Templates](https://egeria-project.org/features/templated-cataloguing/overview/) are used to ensure that as elements are catalogued, they have the right structure, classifications and attachments.

----

In [ ]:

data_lake_schema_template_guid=TEMPLATE_GUIDS['Unity Catalog Schema']
print("dataLakeSchemaTemplateGUID: " + data_lake_schema_template_guid)
print_asset_graph_by_guid(peters_client, data_lake_schema_template_guid)

data_lake_volume_template_guid=TEMPLATE_GUIDS['Unity Catalog Volume']
print("dataLakeVolumeTemplateGUID: " + data_lake_volume_template_guid)
print_asset_graph_by_guid(peters_client, data_lake_volume_template_guid)

data_lake_file_template_guid = peters_client.get_element_guid_by_unique_name("DataLake::{{clinicalTrialId}}::CSVFile::{{filePathName}}")
print("dataLakeFileTemplateGUID: " + data_lake_file_template_guid)
print_asset_graph_by_guid(peters_client, data_lake_file_template_guid)

landing_area_directory_template_guid = peters_client.get_element_guid_by_unique_name("Data Folder::{{fileSystemName}}.{{directoryPathName}}")
print("landingAreaDirectoryTemplateGUID: " + landing_area_directory_template_guid)
print_asset_graph_by_guid(peters_client, landing_area_directory_template_guid)

landing_area_file_template_guid = peters_client.get_element_guid_by_unique_name("LandingArea::{{hospitalName}}::{{clinicalTrialId}}::CSVFile::{{filePathName}}")
print("landingAreaFileTemplateGUID: " + landing_area_file_template_guid)
print_asset_graph_by_guid(peters_client, landing_area_file_template_guid)

#validated_weekly_files_template_guid = "26d6bcdc-ce05-4e0b-8685-cd40777dc5f9"
validated_weekly_files_template_guid = peters_client.get_element_guid_by_unique_name("Data File Collection::{{displayName}}")
print("validatedWeeklyFilesTemplateGUID: " + validated_weekly_files_template_guid)
print_asset_graph_by_guid(peters_client, validated_weekly_files_template_guid)


----

Peter is ready to the process that creates the schema and volume where the data for the clinical trial will reside.  It also configures the templates used when new hospitals are onboarded.

----

In [ ]:

set_up_data_lake_process_name="ClinicalTrials::" + project_identifier + "::set-up-data-lake"

process_guid = peters_client.get_element_guid_by_unique_name(set_up_data_lake_process_name)

process_graph = peters_client.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)



In [ ]:


postgres_server_name="LocalPostgreSQL1"
postgres_secrets_collection_name = "PostgreSQL Server:LocalPostgreSQL1"
postgres_host_identifier="localhost"
postgres_port_number="5432"
postgres_database_name="clinical_trials"

secrets_store_path_name = "loading-bay/secrets/default.omsecrets"

data_lake_directory_path_name="coco-data-lake/research/clinical-trials/" + project_directory_name + "/weekly-measurements"
airflow_dag_name="populate_" + project_schema_name + "_sandbox"



action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "dataLakeCatalog",
      "actionTargetGUID": catalog_guid
    },
    {
      "class" : "NewActionTarget",
      "actionTargetName": "onboardingPipeline",
      "actionTargetGUID": onboarding_pipeline_guid 
    }]

request_parameters = {
     "dataLakeVolumeTemplateGUID" : data_lake_volume_template_guid,
     "dataLakeSchemaTemplateGUID" : data_lake_schema_template_guid,
     "dataLakeFileTemplateGUID" : data_lake_file_template_guid,
     "landingAreaDirectoryTemplateGUID" : landing_area_directory_template_guid,
     "landingAreaFileTemplateGUID" : landing_area_file_template_guid,
     "dataLakeSchemaName" : project_schema_name,
     "dataLakeSchemaDescription" : "Data for the " + project_name + ".",
     "dataLakeVolumeName" : "weekly_measurements",
     "dataLakeVolumeDescription" : "Weekly patient measurements",
     "dataLakeVolumeDirectoryPathName" : data_lake_directory_path_name,
     "validatedWeeklyFilesDataSetName" : "Validated Incoming Weekly Measurements for the " + project_name + ".",
     "validatedWeeklyFilesTemplateGUID" : validated_weekly_files_template_guid,
     "airflowDAGName" : airflow_dag_name,
     "serverName" : postgres_server_name,
     "hostIdentifier" : postgres_host_identifier,
     "portNumber" : postgres_port_number,
     "secretsStorePathName" : secrets_store_path_name,
     "secretsCollectionName" : postgres_secrets_collection_name,
     "versionIdentifier" : "1.0",
     "databaseName" : postgres_database_name,
     "schemaName" : project_schema_name,
     "schemaDescription" : "PostgreSQL database schema for the " + project_name + "."
}

peters_client.initiate_gov_action_process(set_up_data_lake_process_name, None, action_targets, None, request_parameters, None, None)



In [ ]:

display_engine_activity_c(row_limit=1, width=170)


In [ ]:

display_integration_daemon_status(['MaintainLastUpdateDate', 'FilesCataloguer'], paging=True, width=170)


In [ ]:

print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


-----

## Certifying Hospitals (Round One)

Once a hospital has completed all of the requirements, it can be certified for the clinical trial.  This means it can supply data to the clinical trial.  At the start of the clinical trial, Robbie Record, for Oak Dene and Grant Able for Hampton Hospital are ready to go.  Nellie Dunn still does not have the data sharing agreements signed for Old Market.  As a result, only Oak Dene and Hampton hospitals are certified.

----

In [ ]:

certify_hospital_name = "ClinicalTrials::" + project_identifier + "::certify-hospital"
print(certify_hospital_name)

process_guid = tanyas_client.get_element_guid_by_unique_name(certify_hospital_name)

process_graph = tanyas_client.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


In [ ]:
action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": oak_dene_hospital_guid
    }]

tanyas_client.initiate_gov_action_process(certify_hospital_name, None, action_targets, datetime.now(), None, None, None)

In [ ]:
action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": hampton_hospital_guid
    }]

tanyas_client.initiate_gov_action_process(certify_hospital_name, None, action_targets, datetime.now(), None, None, None)

In [ ]:
display_engine_activity_c(row_limit=2, width=170)

----

## Creating the pipelines for each hospital

The clinical trial is about to start, and so Peter creates the pipelines for the three hospitals.

-----

In [ ]:
user_id = "peterprofile"
peters_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = peters_client.create_egeria_bearer_token()

In [ ]:

onboard_hospital_name = "ClinicalTrials::" + project_identifier + "::onboard-hospital"
print(onboard_hospital_name)

process_guid = peters_client.get_element_guid_by_unique_name(onboard_hospital_name)

process_graph = peters_client.get_gov_action_process_graph(process_guid)
print_governance_action_process_graph(process_graph)


In [ ]:

oak_dene_landing_area_directory_name="landing-area/hospitals/oak-dene/clinical-trials/drop-foot"


actionTargets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": oak_dene_hospital_guid
    }]

requestParameters = {
    "landingAreaDirectoryPathName" : oak_dene_landing_area_directory_name
}

peters_client.initiate_gov_action_process(onboard_hospital_name, None, actionTargets, datetime.now(), requestParameters, None, None)


In [ ]:

old_market_landing_area_directory_name="landing-area/hospitals/old-market/clinical-trials/drop-foot"

actionTargets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": old_market_hospital_guid
    }]

requestParameters = {
    "landingAreaDirectoryPathName" : old_market_landing_area_directory_name
}

peters_client.initiate_gov_action_process(onboard_hospital_name, None, actionTargets, datetime.now(), requestParameters, None, None)


In [ ]:

hampton_landing_area_directory_name="landing-area/hospitals/hampton/clinical-trials/drop-foot"

actionTargets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": hampton_hospital_guid
    }]

requestParameters = {
    "landingAreaDirectoryPathName" : hampton_landing_area_directory_name
}

peters_client.initiate_gov_action_process(onboard_hospital_name, None, actionTargets, datetime.now(), requestParameters, None, None)


In [ ]:
display_engine_activity_c(row_limit=3, width=170)

In [ ]:
display_integration_daemon_status(['MaintainLastUpdateDate', 'FilesCataloguer'], paging=True, width=170)

----

## Add data to pipelines

At this point, the processes are set up for Oak Dene Hospital and Hampton Hospital.  It is possible to add the data files to their landing area and they will be systematically added to the data lake folder.   No files are processed for Old Market Hospital.


----

In [ ]:

def add_file_to_landing_area(egeria_client, source_folder, destination_folder, week_number):
    add_file_to_landing_area_name="ClinicalTrials@CocoPharmaceuticals::simulate-ftp"
    source_file_name = source_folder + "/" + "week" + week_number + ".csv"
    print("Moving " + source_file_name + " to " + destination_folder)
    request_parameters = {
        "sourceFile" : source_file_name,
        "destinationDirectory" : destination_folder
    }
    egeria_client.initiate_gov_action_type(add_file_to_landing_area_name, None, None, None, request_parameters, None)

oak_dene_source_folder = '../opt/coco-pharmaceuticals/oak-dene-drop-foot-weekly-measurements'
old_market_source_folder = '../opt/coco-pharmaceuticals/old-market-drop-foot-weekly-measurements'
hampton_source_folder = '../opt/coco-pharmaceuticals/hampton-drop-foot-weekly-measurements'


In [ ]:
add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "1")

In [ ]:
process_design_graph = peters_client.get_gov_action_process_graph(onboarding_pipeline_guid)
print_governance_action_process_graph(process_design_graph)

In [ ]:
oak_dene_onboarding_process_name="Coco::GovernanceActionProcess::" + project_identifier + "::" + oak_dene_hospital_name + "::WeeklyMeasurements::Onboarding"
print(oak_dene_onboarding_process_name)
print_process_instances(peters_client, oak_dene_onboarding_process_name)

In [ ]:
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "1")

In [ ]:
add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "2")
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "2")

In [ ]:
add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "3")

In [ ]:
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "3")

In [ ]:
add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "4")
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "4")

add_file_to_landing_area(peters_client, oak_dene_source_folder, oak_dene_landing_area_directory_name, "5")
add_file_to_landing_area(peters_client, hampton_source_folder, hampton_landing_area_directory_name, "5")

In [ ]:
print_process_instances(peters_client, oak_dene_onboarding_process_name)

In [ ]:
display_engine_activity_c(row_limit=15, width=170)

In [ ]:
print_information_supply_chains(peters_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


----
## Callie looks for status



In [ ]:
user_id = "calliequartile"
callies_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = callies_client.create_egeria_bearer_token()

In [ ]:
print_asset_graph(callies_client, "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot")

In [ ]:
print_asset_graph(callies_client, "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot")

In [ ]:
print_asset_lineage_graph(callies_client, "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot")

In [ ]:

print_asset_graph(callies_client, "DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/teddy-bear-drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv")


In [ ]:

print_asset_graph(callies_client, "DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/teddy-bear-drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv")


In [ ]:

print_asset_lineage_graph(callies_client, "PostgreSQL Relational Database Schema::LocalPostgreSQL1::clinical_trials.teddy_bear_drop_foot")


In [ ]:

#print_anchor_graph_by_guid(callies_client, '8a921039-ad5f-454d-ae17-e5a5b69f9333')
print_related_elements_for_name(callies_client, 'CertificationType:: Valid data for Teddy Bear Drop Foot Clinical Trial', "Certification")


In [ ]:

#print_anchor_graph_by_guid(callies_client, '8a921039-ad5f-454d-ae17-e5a5b69f9333')
print_anchor_graph(callies_client, 'CertificationType:: Valid data for Teddy Bear Drop Foot Clinical Trial')


In [ ]:
print_asset_graph(callies_client, "Data File Collection::Validated Incoming Weekly Measurements for the Teddy Bear Drop Foot Clinical Trial.")

In [ ]:

print_asset_lineage_graph(callies_client, "Data File Collection::Validated Incoming Weekly Measurements for the Teddy Bear Drop Foot Clinical Trial.")


In [ ]:

print_asset_lineage_graph(callies_client, "DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv")


In [ ]:

print_solution_blueprints(callies_client, "Clinical Trial Management Solution Blueprint")


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation::" + project_identifier, False)


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation::" + project_identifier, True, False)


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation::" + project_identifier, True, True)


In [ ]:

print_information_supply_chains(callies_client, "Physical", True, True)


In [ ]:
print_asset_graph(callies_client, onboard_hospital_name)

In [ ]:

print_solution_components(callies_client, "Pipeline")


In [ ]:

print_solution_components(callies_client, "Treatment Validation Sandbox")


In [ ]:

print_solution_components(callies_client, "Treatment")


In [ ]:

print_solution_components(callies_client, "Populate")


In [ ]:
search_results = callies_client.find_in_asset_domain("Validated")

print_search_results(search_results)

In [ ]:
search_results = callies_client.find_in_asset_domain("LandingArea::Hampton")

print_search_results(search_results)

In [ ]:
search_results = callies_client.find_in_asset_domain("DataLake::" + project_identifier + "::CSVFile::")

print_search_results(search_results)

In [ ]:
search_results = callies_client.find_in_asset_domain(project_identifier + "_000003.csv")

print_search_results(search_results)

In [ ]:
print_asset_graph(callies_client, "Coco:GovernanceActionProcess:ClinicalTrials:WeeklyMeasurements:Onboarding")

In [ ]:
print_asset_graph(callies_client, "Apache Airflow DAG::" + airflow_dag_name)

In [ ]:

print_asset_lineage_graph(callies_client, "Apache Airflow DAG:" + airflow_dag_name)


## Certifying Old Market Hospital

After some prompting, Nellie gets the data sharing agreement signed and Tanya is able to certify Old Market Hospital.

----

In [ ]:
user_id = "tanyatidie"
tanyas_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = tanyas_client.create_egeria_bearer_token()

In [ ]:

action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": old_market_hospital_guid
    }]

tanyas_client.initiate_gov_action_process(certify_hospital_name, None, action_targets, datetime.now(), None, None, None)

----

Peter is then able to create the onboarding pipeline for Old Market Hospital

----

In [ ]:
user_id = "peterprofile"
peters_client = EgeriaTech(view_server, url, user_id, user_pwd)
token = peters_client.create_egeria_bearer_token()

In [ ]:

action_targets = [{
      "class" : "NewActionTarget",
      "actionTargetName": "hospital",
      "actionTargetGUID": old_market_hospital_guid
    }]

request_parameters = {
    "landingAreaDirectoryPathName" : old_market_landing_area_directory_name
}

peters_client.initiate_gov_action_process(onboard_hospital_name, None, action_targets, datetime.now(), request_parameters, None, None)


----

Now any files that are added to Old Market Hospital's landing area are ingested into the data lake.

----

In [ ]:
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "1")

In [ ]:
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "2")

In [ ]:
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "3")

In [ ]:
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "4")
add_file_to_landing_area(peters_client, old_market_source_folder, old_market_landing_area_directory_name, "5")

In [ ]:
display_engine_activity_c(row_limit=5, width=170)

----
## After all hospitals are registered ...

---

In [ ]:

print_asset_lineage_graph(callies_client, "PostgreSQL Relational Database Schema:LocalPostgreSQL1:clinical_trials.drop_foot")


In [ ]:

print_information_supply_chains(callies_client, "Clinical Trial Treatment Validation:PROJ-CT-TBDF", add_implementation=True, for_lineage=True)


In [ ]:

mermaid_graph="---\ntitle: Information Supply Chain - Clinical Trial Treatment Validation:PROJ-CT-TBDF [6859e374-8880-473a-b90e-d9f55935d5dd]\n---\nflowchart TD\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\nsubgraph Details\nsubgraph Segments\n7edca02c-e726-4570-815c-280bdf5498b9@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Data Lake to Sandbox**\"}\n7edca02c-e726-4570-815c-280bdf5498b9==>|\"assemble,\n[Information Supply Chain Link]\"|e4303326-e418-4f77-b8e7-fd5d34717594\n4fc47e60-f1b5-469c-b666-e1f4570e749f@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Deliver Treatment Assessment Report**\"}\ne4303326-e418-4f77-b8e7-fd5d34717594@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Assess Treatment under Trial**\"}\ne4303326-e418-4f77-b8e7-fd5d34717594==>|\"publish results,\n[Information Supply Chain Link]\"|4fc47e60-f1b5-469c-b666-e1f4570e749f\n38635d38-f728-400d-a8ec-7c26e68b7c0f@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Weekly Measurements Onboarding**\"}\n38635d38-f728-400d-a8ec-7c26e68b7c0f==>|\"capture and catalog,\n[Information Supply Chain Link]\"|7edca02c-e726-4570-815c-280bdf5498b9\n04ae768e-3816-47bc-bddb-c9ae25018684@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Hospital Delivers Patient Weekly Readings**\"}\n04ae768e-3816-47bc-bddb-c9ae25018684==>|\"publish,\n[Information Supply Chain Link]\"|38635d38-f728-400d-a8ec-7c26e68b7c0f\nend\nsubgraph Design\na5d4d638-6836-47e5-99d0-fdcde637e13f@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Weekly Measurements Data Lake Folder::V1.0**\"}\n26c07ca4-3b8e-484b-812b-36c1ace4b275@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Populate Sandbox::V1.0**\"}\nd48f579f-76d3-4c49-b1b4-575f5645a9d0@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Treatment Validation Sandbox::V1.0**\"}\n72a86eec-9734-4bc0-babb-4fec0aa7c9ff@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Assemble Treatment Assessment Report::V1.0**\"}\n48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Treatment Efficacy Evidence::V1.0**\"}\n0bf2547c-937c-41b6-814f-6284849271a1@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Treatment Assessment Report Validation and Delivery::V1.0**\"}\nb5c8da4c-f925-4cf1-8294-e43cd2c1a584@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Analyse Patient Data::V1.0**\"}\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Hospital Landing Area Folder::V1.0**\"}\n7f5dca65-50b4-4103-9ac7-3a406a09047a@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Weekly Measurements Onboarding Pipeline::V1.0**\"}\n07705e15-efff-4f80-8992-f04ac85e0ef1@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Landing Folder Cataloguer::V1.0**\"}\nee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Hospital Processes::V1.0**\"}\n7edca02c-e726-4570-815c-280bdf5498b9-->|\"Supporting Supply Chain,\n[Implemented By]\"|a5d4d638-6836-47e5-99d0-fdcde637e13f\n7edca02c-e726-4570-815c-280bdf5498b9-->|\"Supporting Supply Chain,\n[Implemented By]\"|26c07ca4-3b8e-484b-812b-36c1ace4b275\n7edca02c-e726-4570-815c-280bdf5498b9-->|\"Supporting Supply Chain,\n[Implemented By]\"|d48f579f-76d3-4c49-b1b4-575f5645a9d0\n7f5dca65-50b4-4103-9ac7-3a406a09047a==>|\"save new files,\n[Solution Linking Wire]\"|a5d4d638-6836-47e5-99d0-fdcde637e13f\na5d4d638-6836-47e5-99d0-fdcde637e13f==>|\"read certified files,\n[Solution Linking Wire]\"|26c07ca4-3b8e-484b-812b-36c1ace4b275\n26c07ca4-3b8e-484b-812b-36c1ace4b275==>|\"write patient measurements,\n[Solution Linking Wire]\"|d48f579f-76d3-4c49-b1b4-575f5645a9d0\nd48f579f-76d3-4c49-b1b4-575f5645a9d0==>|\"retrieve patient data,\n[Solution Linking Wire]\"|b5c8da4c-f925-4cf1-8294-e43cd2c1a584\n4fc47e60-f1b5-469c-b666-e1f4570e749f-->|\"Supporting Supply Chain,\n[Implemented By]\"|72a86eec-9734-4bc0-babb-4fec0aa7c9ff\n4fc47e60-f1b5-469c-b666-e1f4570e749f-->|\"Supporting Supply Chain,\n[Implemented By]\"|48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5\n4fc47e60-f1b5-469c-b666-e1f4570e749f-->|\"Supporting Supply Chain,\n[Implemented By]\"|0bf2547c-937c-41b6-814f-6284849271a1\n48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5==>|\"retrieve evidence,\n[Solution Linking Wire]\"|72a86eec-9734-4bc0-babb-4fec0aa7c9ff\n72a86eec-9734-4bc0-babb-4fec0aa7c9ff==>|\"publish results,\n[Solution Linking Wire]\"|0bf2547c-937c-41b6-814f-6284849271a1\nb5c8da4c-f925-4cf1-8294-e43cd2c1a584==>|\"publish results,\n[Solution Linking Wire]\"|48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5\ne4303326-e418-4f77-b8e7-fd5d34717594-->|\"Supporting Supply Chain,\n[Implemented By]\"|48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5\ne4303326-e418-4f77-b8e7-fd5d34717594-->|\"Supporting Supply Chain,\n[Implemented By]\"|b5c8da4c-f925-4cf1-8294-e43cd2c1a584\ne4303326-e418-4f77-b8e7-fd5d34717594-->|\"Supporting Supply Chain,\n[Implemented By]\"|d48f579f-76d3-4c49-b1b4-575f5645a9d0\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|1c150d6e-30cf-481c-9afb-3b06c9c9e78f\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|7f5dca65-50b4-4103-9ac7-3a406a09047a\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|07705e15-efff-4f80-8992-f04ac85e0ef1\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|a5d4d638-6836-47e5-99d0-fdcde637e13f\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f==>|\"detect new files,\n[Solution Linking Wire]\"|07705e15-efff-4f80-8992-f04ac85e0ef1\nee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec==>|\"publish,\n[Solution Linking Wire]\"|1c150d6e-30cf-481c-9afb-3b06c9c9e78f\n07705e15-efff-4f80-8992-f04ac85e0ef1==>|\"request onboarding,\n[Solution Linking Wire]\"|7f5dca65-50b4-4103-9ac7-3a406a09047a\n04ae768e-3816-47bc-bddb-c9ae25018684-->|\"Supporting Supply Chain,\n[Implemented By]\"|1c150d6e-30cf-481c-9afb-3b06c9c9e78f\n04ae768e-3816-47bc-bddb-c9ae25018684-->|\"Supporting Supply Chain,\n[Implemented By]\"|ee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec\nend\nsubgraph Implementation\nd367d7cb-bc7f-4572-b883-895052f07935@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week3.csv_patientId_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\nea9cc426-f108-4365-84be-0e2f95b52b2c@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000012.csv_patientId_column**\"}\nd367d7cb-bc7f-4572-b883-895052f07935==>|\"copy [Data Mapping]\"|ea9cc426-f108-4365-84be-0e2f95b52b2c\n16a76505-cca4-4af7-8ac5-8143ab3f65d4@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week4.csv_date_column_archivedOn_Wed Apr 23 20:59:08 BST 2025**\"}\n9f6a9170-5e20-4e37-82c8-80f21d901181@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000013.csv_patientId_column**\"}\n16a76505-cca4-4af7-8ac5-8143ab3f65d4==>|\"copy [Data Mapping]\"|9f6a9170-5e20-4e37-82c8-80f21d901181\n312d3284-8962-4ea2-9c20-3ce1eb9b9d88@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_angleLeft_column_archivedOn_Wed Apr 23 20:52:58 BST 2025**\"}\n4a988bae-30cb-4cac-a375-d8e8aa25b893@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000002.csv_patientId_column**\"}\n312d3284-8962-4ea2-9c20-3ce1eb9b9d88==>|\"copy [Data Mapping]\"|4a988bae-30cb-4cac-a375-d8e8aa25b893\n85bf997e-0548-43c9-bdc7-20eaf2e82878@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week2.csv_angleRight_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\neb20ddb9-32fa-47c4-883e-e737d4e25320@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000011.csv_patientId_column**\"}\n85bf997e-0548-43c9-bdc7-20eaf2e82878==>|\"copy [Data Mapping]\"|eb20ddb9-32fa-47c4-883e-e737d4e25320\n3dadf467-46a3-4b9a-aad9-87ba79418715@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_angleLeft_column_archivedOn_Wed Apr 23 20:52:41 BST 2025**\"}\nc26a1f9f-2686-4697-b1d9-28f6dd416149@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv_patientId_column**\"}\n3dadf467-46a3-4b9a-aad9-87ba79418715==>|\"copy [Data Mapping]\"|c26a1f9f-2686-4697-b1d9-28f6dd416149\nc192ad79-3fd5-4e07-a2d0-276dc2050481@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_angleLeft_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\n47c3142d-2e72-4d97-b495-324ff303bb1c@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000004.csv_patientId_column**\"}\nc192ad79-3fd5-4e07-a2d0-276dc2050481==>|\"copy [Data Mapping]\"|47c3142d-2e72-4d97-b495-324ff303bb1c\nce5d56af-7475-4030-9d88-d4ed7a485c38@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_date_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\na0688d35-64e0-4b46-b9b7-a91b6977c5b2@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000003.csv_patientId_column**\"}\nce5d56af-7475-4030-9d88-d4ed7a485c38==>|\"copy [Data Mapping]\"|a0688d35-64e0-4b46-b9b7-a91b6977c5b2\naa16f792-e97e-4bd0-abfb-029059d4c8a9@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_angleRight_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\naa16f792-e97e-4bd0-abfb-029059d4c8a9==>|\"copy [Data Mapping]\"|a0688d35-64e0-4b46-b9b7-a91b6977c5b2\nbc21e09c-7664-4555-b481-37e0b9ddd665@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_angleRight_column_archivedOn_Wed Apr 23 20:53:04 BST 2025**\"}\n64bfac1b-c502-454a-92cd-f410d643dfd8@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000009.csv_patientId_column**\"}\nbc21e09c-7664-4555-b481-37e0b9ddd665==>|\"copy [Data Mapping]\"|64bfac1b-c502-454a-92cd-f410d643dfd8\ne19b2f8c-2513-4689-befd-0fdfddacea3f@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week2.csv_date_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\ne19b2f8c-2513-4689-befd-0fdfddacea3f==>|\"copy [Data Mapping]\"|eb20ddb9-32fa-47c4-883e-e737d4e25320\nb3773e8d-d562-434d-a455-a567723c48ac@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_patientId_column_archivedOn_Wed Apr 23 20:53:02 BST 2025**\"}\n87876c24-f38e-4f0f-87fa-08b59e2ace15@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000007.csv_patientId_column**\"}\nb3773e8d-d562-434d-a455-a567723c48ac==>|\"copy [Data Mapping]\"|87876c24-f38e-4f0f-87fa-08b59e2ace15\n5b0cf874-3f00-4440-9d3b-15a3a88ed84d@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_angleRight_column_archivedOn_Wed Apr 23 20:53:03 BST 2025**\"}\nad82f669-a81d-440f-9a6f-586e4107266d@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000008.csv_patientId_column**\"}\n5b0cf874-3f00-4440-9d3b-15a3a88ed84d==>|\"copy [Data Mapping]\"|ad82f669-a81d-440f-9a6f-586e4107266d\n93e81b66-bd51-408b-9dab-4ffc61cda040@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_date_column_archivedOn_Wed Apr 23 20:53:02 BST 2025**\"}\n93e81b66-bd51-408b-9dab-4ffc61cda040==>|\"copy [Data Mapping]\"|87876c24-f38e-4f0f-87fa-08b59e2ace15\n5d6faf77-bca0-473a-9da4-1f8f5e203f75@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week5.csv_angleLeft_column_archivedOn_Wed Apr 23 20:59:13 BST 2025**\"}\na1065c6a-e8dc-48be-a0ce-daab251b8279@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000014.csv_patientId_column**\"}\n5d6faf77-bca0-473a-9da4-1f8f5e203f75==>|\"copy [Data Mapping]\"|a1065c6a-e8dc-48be-a0ce-daab251b8279\n985104b8-bf32-41e2-91fa-0b354f8861d8@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week1.csv_angleLeft_column_archivedOn_Wed Apr 23 20:59:05 BST 2025**\"}\n82f10cb5-7625-4fdd-a72a-f2a388ab871d@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000010.csv_patientId_column**\"}\n985104b8-bf32-41e2-91fa-0b354f8861d8==>|\"copy [Data Mapping]\"|82f10cb5-7625-4fdd-a72a-f2a388ab871d\na4fcebd2-f475-40f9-839e-17a1bbe879c8@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_patientId_column_archivedOn_Wed Apr 23 20:52:41 BST 2025**\"}\na4fcebd2-f475-40f9-839e-17a1bbe879c8==>|\"copy [Data Mapping]\"|c26a1f9f-2686-4697-b1d9-28f6dd416149\n2a3b2133-7705-4dd3-96f4-59e68c0c584a@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_angleRight_column_archivedOn_Wed Apr 23 20:53:02 BST 2025**\"}\n2a3b2133-7705-4dd3-96f4-59e68c0c584a==>|\"copy [Data Mapping]\"|87876c24-f38e-4f0f-87fa-08b59e2ace15\nc7b3da12-4d8f-4a61-b182-10e7ba92bac1@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_date_column_archivedOn_Wed Apr 23 20:53:04 BST 2025**\"}\nc7b3da12-4d8f-4a61-b182-10e7ba92bac1==>|\"copy [Data Mapping]\"|64bfac1b-c502-454a-92cd-f410d643dfd8\ndcad51d6-66da-49ef-b48e-0db24b27a079@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week4.csv_angleLeft_column_archivedOn_Wed Apr 23 20:59:08 BST 2025**\"}\ndcad51d6-66da-49ef-b48e-0db24b27a079==>|\"copy [Data Mapping]\"|9f6a9170-5e20-4e37-82c8-80f21d901181\nb37b8106-c884-494d-9aed-10e1df027e2e@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_angleRight_column_archivedOn_Wed Apr 23 20:53:01 BST 2025**\"}\n750b9fb3-aae2-4d14-b500-e136dfd48d1e@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000006.csv_patientId_column**\"}\nb37b8106-c884-494d-9aed-10e1df027e2e==>|\"copy [Data Mapping]\"|750b9fb3-aae2-4d14-b500-e136dfd48d1e\n68d912f0-9c3c-469b-8f45-9f8f6012b18f@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_angleLeft_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\n68d912f0-9c3c-469b-8f45-9f8f6012b18f==>|\"copy [Data Mapping]\"|a0688d35-64e0-4b46-b9b7-a91b6977c5b2\n697d48ea-459c-4a16-840e-ecec8dfc2ca9@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_date_column_archivedOn_Wed Apr 23 20:52:58 BST 2025**\"}\n697d48ea-459c-4a16-840e-ecec8dfc2ca9==>|\"copy [Data Mapping]\"|4a988bae-30cb-4cac-a375-d8e8aa25b893\ne30903c5-d863-4ea8-941f-9dde8c0d1472@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week1.csv_angleRight_column_archivedOn_Wed Apr 23 20:59:05 BST 2025**\"}\ne30903c5-d863-4ea8-941f-9dde8c0d1472==>|\"copy [Data Mapping]\"|82f10cb5-7625-4fdd-a72a-f2a388ab871d\n9c5ad26e-0cfc-4f32-a64a-f4e9f0cc8526@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_angleRight_column_archivedOn_Wed Apr 23 20:52:41 BST 2025**\"}\n9c5ad26e-0cfc-4f32-a64a-f4e9f0cc8526==>|\"copy [Data Mapping]\"|c26a1f9f-2686-4697-b1d9-28f6dd416149\n1c1de4b7-4c19-4f88-a40c-ea650eba3629@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week3.csv_date_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\n1c1de4b7-4c19-4f88-a40c-ea650eba3629==>|\"copy [Data Mapping]\"|ea9cc426-f108-4365-84be-0e2f95b52b2c\nbe171af3-db8d-41f2-91ab-db1edf0aad0d@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_angleLeft_column_archivedOn_Wed Apr 23 20:53:00 BST 2025**\"}\n5f28ccec-d4fc-41cd-826e-90fa203953c9@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000005.csv_patientId_column**\"}\nbe171af3-db8d-41f2-91ab-db1edf0aad0d==>|\"copy [Data Mapping]\"|5f28ccec-d4fc-41cd-826e-90fa203953c9\nab4a83f8-924f-4696-aa62-a74f65f3fe17@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_date_column_archivedOn_Wed Apr 23 20:53:00 BST 2025**\"}\nab4a83f8-924f-4696-aa62-a74f65f3fe17==>|\"copy [Data Mapping]\"|5f28ccec-d4fc-41cd-826e-90fa203953c9\n91e5f6e9-d836-44bf-a88a-64df4e30f48d@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week4.csv_angleRight_column_archivedOn_Wed Apr 23 20:59:08 BST 2025**\"}\n91e5f6e9-d836-44bf-a88a-64df4e30f48d==>|\"copy [Data Mapping]\"|9f6a9170-5e20-4e37-82c8-80f21d901181\n479728fe-c9b6-4349-a4c8-8e87b2190642@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_date_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\n479728fe-c9b6-4349-a4c8-8e87b2190642==>|\"copy [Data Mapping]\"|47c3142d-2e72-4d97-b495-324ff303bb1c\ncd5ada12-c854-4818-9bf9-e48584a041b8@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_angleLeft_column_archivedOn_Wed Apr 23 20:53:03 BST 2025**\"}\ncd5ada12-c854-4818-9bf9-e48584a041b8==>|\"copy [Data Mapping]\"|ad82f669-a81d-440f-9a6f-586e4107266d\nc93b3371-ec01-41c5-b912-339a04f10de1@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_date_column_archivedOn_Wed Apr 23 20:53:03 BST 2025**\"}\nc93b3371-ec01-41c5-b912-339a04f10de1==>|\"copy [Data Mapping]\"|ad82f669-a81d-440f-9a6f-586e4107266d\nba037aed-a61e-4bfc-91e7-de95f7fa3e2f@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week1.csv_patientId_column_archivedOn_Wed Apr 23 20:59:05 BST 2025**\"}\nba037aed-a61e-4bfc-91e7-de95f7fa3e2f==>|\"copy [Data Mapping]\"|82f10cb5-7625-4fdd-a72a-f2a388ab871d\n3d642257-8ef7-4ef1-822a-855e4fca1162@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_date_column_archivedOn_Wed Apr 23 20:52:41 BST 2025**\"}\n3d642257-8ef7-4ef1-822a-855e4fca1162==>|\"copy [Data Mapping]\"|c26a1f9f-2686-4697-b1d9-28f6dd416149\ne07be6ba-3f54-406a-bd01-dcf994a3000e@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_patientId_column_archivedOn_Wed Apr 23 20:53:03 BST 2025**\"}\ne07be6ba-3f54-406a-bd01-dcf994a3000e==>|\"copy [Data Mapping]\"|ad82f669-a81d-440f-9a6f-586e4107266d\n3677d8ff-d3ec-4a23-9296-1b026daa80bb@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_angleLeft_column_archivedOn_Wed Apr 23 20:53:04 BST 2025**\"}\n3677d8ff-d3ec-4a23-9296-1b026daa80bb==>|\"copy [Data Mapping]\"|64bfac1b-c502-454a-92cd-f410d643dfd8\n948df020-b9f1-472c-92d8-44699be0022e@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_angleLeft_column_archivedOn_Wed Apr 23 20:53:02 BST 2025**\"}\n948df020-b9f1-472c-92d8-44699be0022e==>|\"copy [Data Mapping]\"|87876c24-f38e-4f0f-87fa-08b59e2ace15\n3d2d1b40-fb0d-42ee-82bc-80466ac6437d@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_angleRight_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\n3d2d1b40-fb0d-42ee-82bc-80466ac6437d==>|\"copy [Data Mapping]\"|47c3142d-2e72-4d97-b495-324ff303bb1c\n9032c840-ea1e-464c-9050-d95669d96f92@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week2.csv_patientId_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\n9032c840-ea1e-464c-9050-d95669d96f92==>|\"copy [Data Mapping]\"|eb20ddb9-32fa-47c4-883e-e737d4e25320\n7af2fa96-ab8e-4e0f-beef-3b3406a23031@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week1.csv_date_column_archivedOn_Wed Apr 23 20:59:05 BST 2025**\"}\n7af2fa96-ab8e-4e0f-beef-3b3406a23031==>|\"copy [Data Mapping]\"|82f10cb5-7625-4fdd-a72a-f2a388ab871d\nb418a842-3907-40d1-880d-f4bdc465a285@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_patientId_column_archivedOn_Wed Apr 23 20:53:01 BST 2025**\"}\nb418a842-3907-40d1-880d-f4bdc465a285==>|\"copy [Data Mapping]\"|750b9fb3-aae2-4d14-b500-e136dfd48d1e\n8f9d3489-cca9-442a-bbe7-1486560fdfb7@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_angleRight_column_archivedOn_Wed Apr 23 20:53:00 BST 2025**\"}\n8f9d3489-cca9-442a-bbe7-1486560fdfb7==>|\"copy [Data Mapping]\"|5f28ccec-d4fc-41cd-826e-90fa203953c9\n6f9f66fd-2e41-4b7d-825d-124288cad6a4@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_date_column_archivedOn_Wed Apr 23 20:52:57 BST 2025**\"}\n7de8760e-5630-4f74-b92d-6534d37713a8@{ shape: rounded, label: \"*Tabular Column*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv_patientId_column**\"}\n6f9f66fd-2e41-4b7d-825d-124288cad6a4==>|\"copy [Data Mapping]\"|7de8760e-5630-4f74-b92d-6534d37713a8\n1ce9095b-7fe6-410f-ac0e-78e3cac5814a@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_angleRight_column_archivedOn_Wed Apr 23 20:52:57 BST 2025**\"}\n1ce9095b-7fe6-410f-ac0e-78e3cac5814a==>|\"copy [Data Mapping]\"|7de8760e-5630-4f74-b92d-6534d37713a8\n3e87d13b-2d86-48df-8073-3b24b945dd80@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_patientId_column_archivedOn_Wed Apr 23 20:52:58 BST 2025**\"}\n3e87d13b-2d86-48df-8073-3b24b945dd80==>|\"copy [Data Mapping]\"|4a988bae-30cb-4cac-a375-d8e8aa25b893\nc7f82794-9ebd-4ba1-b29f-1b3b224551a7@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_angleLeft_column_archivedOn_Wed Apr 23 20:52:57 BST 2025**\"}\nc7f82794-9ebd-4ba1-b29f-1b3b224551a7==>|\"copy [Data Mapping]\"|7de8760e-5630-4f74-b92d-6534d37713a8\nb0815266-f62c-4491-b516-3bca365bbb62@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_angleRight_column_archivedOn_Wed Apr 23 20:52:58 BST 2025**\"}\nb0815266-f62c-4491-b516-3bca365bbb62==>|\"copy [Data Mapping]\"|4a988bae-30cb-4cac-a375-d8e8aa25b893\n6feeace9-5620-42e6-b36e-5916ee7e8b41@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_patientId_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\n6feeace9-5620-42e6-b36e-5916ee7e8b41==>|\"copy [Data Mapping]\"|a0688d35-64e0-4b46-b9b7-a91b6977c5b2\n4d2582e0-3ad2-4c70-add5-ce078b0fd3b8@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_patientId_column_archivedOn_Wed Apr 23 20:52:57 BST 2025**\"}\n4d2582e0-3ad2-4c70-add5-ce078b0fd3b8==>|\"copy [Data Mapping]\"|7de8760e-5630-4f74-b92d-6534d37713a8\nb9757c30-75a3-4460-b4c0-e9423770dfd1@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_patientId_column_archivedOn_Wed Apr 23 20:53:04 BST 2025**\"}\nb9757c30-75a3-4460-b4c0-e9423770dfd1==>|\"copy [Data Mapping]\"|64bfac1b-c502-454a-92cd-f410d643dfd8\n2480ea7f-c268-435c-a475-06428798effd@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week4.csv_patientId_column_archivedOn_Wed Apr 23 20:59:08 BST 2025**\"}\n2480ea7f-c268-435c-a475-06428798effd==>|\"copy [Data Mapping]\"|9f6a9170-5e20-4e37-82c8-80f21d901181\ncfb00a09-801a-44fc-a29f-5e725919bee6@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_date_column_archivedOn_Wed Apr 23 20:53:01 BST 2025**\"}\ncfb00a09-801a-44fc-a29f-5e725919bee6==>|\"copy [Data Mapping]\"|750b9fb3-aae2-4d14-b500-e136dfd48d1e\n7ba8acda-5c6b-4f48-a381-a0e14382a08d@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week5.csv_patientId_column_archivedOn_Wed Apr 23 20:59:13 BST 2025**\"}\n7ba8acda-5c6b-4f48-a381-a0e14382a08d==>|\"copy [Data Mapping]\"|a1065c6a-e8dc-48be-a0ce-daab251b8279\ne96d69ea-5a07-4d77-9862-19a53ac71bf8@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week5.csv_angleRight_column_archivedOn_Wed Apr 23 20:59:13 BST 2025**\"}\ne96d69ea-5a07-4d77-9862-19a53ac71bf8==>|\"copy [Data Mapping]\"|a1065c6a-e8dc-48be-a0ce-daab251b8279\n4ce00c3c-c037-4b80-a56d-bb25ca40619e@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_angleLeft_column_archivedOn_Wed Apr 23 20:53:01 BST 2025**\"}\n4ce00c3c-c037-4b80-a56d-bb25ca40619e==>|\"copy [Data Mapping]\"|750b9fb3-aae2-4d14-b500-e136dfd48d1e\ne8ac318b-885c-4877-a353-c7b7c63f73b1@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_patientId_column_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\ne8ac318b-885c-4877-a353-c7b7c63f73b1==>|\"copy [Data Mapping]\"|47c3142d-2e72-4d97-b495-324ff303bb1c\n9cea142e-2a6f-4e2e-8a4a-5148490e7de1@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_patientId_column_archivedOn_Wed Apr 23 20:53:00 BST 2025**\"}\n9cea142e-2a6f-4e2e-8a4a-5148490e7de1==>|\"copy [Data Mapping]\"|5f28ccec-d4fc-41cd-826e-90fa203953c9\n6c854386-b00e-4a2d-906b-9e3d937cd20c@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week3.csv_angleRight_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\n6c854386-b00e-4a2d-906b-9e3d937cd20c==>|\"copy [Data Mapping]\"|ea9cc426-f108-4365-84be-0e2f95b52b2c\n9d5ee104-38b6-435c-96b4-37183cca58f8@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week5.csv_date_column_archivedOn_Wed Apr 23 20:59:13 BST 2025**\"}\n9d5ee104-38b6-435c-96b4-37183cca58f8==>|\"copy [Data Mapping]\"|a1065c6a-e8dc-48be-a0ce-daab251b8279\n978ee626-c755-4e99-ac94-a1d295fc8f68@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week2.csv_angleLeft_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\n978ee626-c755-4e99-ac94-a1d295fc8f68==>|\"copy [Data Mapping]\"|eb20ddb9-32fa-47c4-883e-e737d4e25320\n4648bd1a-a9dc-4ee1-98e0-7e6534e3c929@{ shape: notch-pent, label: \"*Tabular Column*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week3.csv_angleLeft_column_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\n4648bd1a-a9dc-4ee1-98e0-7e6534e3c929==>|\"copy [Data Mapping]\"|ea9cc426-f108-4365-84be-0e2f95b52b2c\n7e296f64-2257-44fd-8bc7-6312bb6d7af6@{ shape: rounded, label: \"*Data File Collection*\n**Data File Collection:Validated Incoming Weekly Measurements For Teddy Bear Drop Foot Clinical Trial**\"}\n79b79758-1eaf-4634-989f-6c1b06678402@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000009.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|79b79758-1eaf-4634-989f-6c1b06678402\nf58d9f5e-a019-4c4f-8c5b-cb173709c604@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000006.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|f58d9f5e-a019-4c4f-8c5b-cb173709c604\n28a9cb83-2fea-4f52-9c7f-890a1724c352@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000013.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|28a9cb83-2fea-4f52-9c7f-890a1724c352\nf92fdc60-e513-4724-8831-f198de24f477@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000003.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|f92fdc60-e513-4724-8831-f198de24f477\n3d2f0014-b01a-4254-8c08-5249f99c4fac@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000014.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|3d2f0014-b01a-4254-8c08-5249f99c4fac\nb25d01d9-459f-4d2a-8eb2-400109fc07e2@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000011.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|b25d01d9-459f-4d2a-8eb2-400109fc07e2\n53a51fab-cb88-4cc1-916d-a378ab9aa03f@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000010.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|53a51fab-cb88-4cc1-916d-a378ab9aa03f\ne69f7e36-0a33-4559-b1a3-d864a7118814@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000008.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|e69f7e36-0a33-4559-b1a3-d864a7118814\n666a520c-8e33-4cd0-9147-9e4674a0d678@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000004.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|666a520c-8e33-4cd0-9147-9e4674a0d678\nd450fa0b-c432-47cd-948a-9e546794f3a7@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|d450fa0b-c432-47cd-948a-9e546794f3a7\nf7eca8b6-ac11-4e90-bb38-8fc0075880d5@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000005.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|f7eca8b6-ac11-4e90-bb38-8fc0075880d5\n745e39e9-04ef-4a0f-afdc-03a4c5f4448f@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000002.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|745e39e9-04ef-4a0f-afdc-03a4c5f4448f\n929f82e1-97e7-4c1f-b8ed-9764567c1717@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000007.csv**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Data Set Content\"|929f82e1-97e7-4c1f-b8ed-9764567c1717\n16299e50-a351-4ad2-8027-898ec8a3176d@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService62c1f8f3-3159-4dc4-ba3f-7b78e0a35875**\"}\n1b34b404-180d-4e35-bec0-c43b563425fe@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000012.csv**\"}\n16299e50-a351-4ad2-8027-898ec8a3176d==>|\"Data Flow\"|1b34b404-180d-4e35-bec0-c43b563425fe\ne11831d8-d861-4358-937a-f4f1b7be3a94@{ shape: rounded, label: \"*Deployed Software Component*\n**Apache Airflow DAG:populateDropfootSandbox**\"}\n4b6722d1-9168-4f1f-b2a4-26bc13b51512@{ shape: rounded, label: \"*Deployed Database Schema*\n**PostgreSQL Relational Database Schema:LocalPostgreSQL1:clinical_trials.drop_foot**\"}\ne11831d8-d861-4358-937a-f4f1b7be3a94==>|\"Save Certified Data [Data Flow]\"|4b6722d1-9168-4f1f-b2a4-26bc13b51512\nb2b8ade5-018d-4f15-afcd-e31c2a993335@{ shape: rounded, label: \"*Data Folder*\n**Data Folder:.landing-area/hospitals/old-market/clinical-trials/drop-foot**\"}\n1b98cdac-dd0a-4621-93db-99ef5a1098bc@{ shape: rounded, label: \"*Integration Connector*\n**Egeria:IntegrationGroup:Default:GeneralFilesMonitorIntegrationConnector**\"}\nb2b8ade5-018d-4f15-afcd-e31c2a993335==>|\"new file notification [Data Flow]\"|1b98cdac-dd0a-4621-93db-99ef5a1098bc\n6703dd61-76fe-4741-b55a-967a4eb2bda3@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week1.csv_archivedOn_Wed Apr 23 20:59:05 BST 2025**\"}\n628b1e7f-64d0-4305-8a10-aa3204ac338d@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService551f01c3-f2e2-4a0d-a39c-d4640c623e89**\"}\n6703dd61-76fe-4741-b55a-967a4eb2bda3==>|\"Data Flow\"|628b1e7f-64d0-4305-8a10-aa3204ac338d\n042b00c9-7614-466b-b501-f6c90ed34b51@{ shape: rounded, label: \"*Governance Action Process*\n**Coco:GovernanceActionProcess:PROJ-CT-TBDF:Hampton Hospital:WeeklyMeasurements:Onboarding**\"}\n1b98cdac-dd0a-4621-93db-99ef5a1098bc==>|\"new file trigger [Data Flow]\"|042b00c9-7614-466b-b501-f6c90ed34b51\n0a9c3d3d-8f37-4487-aeae-d20bee32d16a@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_archivedOn_Wed Apr 23 20:53:04 BST 2025**\"}\n37b81f25-beea-44e2-83a6-c5d41724e026@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService3e3b914f-9f98-41c5-a26c-f7dc0c679d19**\"}\n0a9c3d3d-8f37-4487-aeae-d20bee32d16a==>|\"Data Flow\"|37b81f25-beea-44e2-83a6-c5d41724e026\nf5626f07-5a4f-4ad6-801e-c66f9415a885@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_archivedOn_Wed Apr 23 20:53:00 BST 2025**\"}\n19fcbdb8-eff0-42c9-98ed-e99ee8d9e3f7@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServicec3055fc8-2f1b-4238-a57b-91949e612006**\"}\nf5626f07-5a4f-4ad6-801e-c66f9415a885==>|\"Data Flow\"|19fcbdb8-eff0-42c9-98ed-e99ee8d9e3f7\n42c5ef2f-035d-4cf8-9897-b5da133f8103@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServiceedb3062f-451d-4b47-9e37-92a9fdad0888**\"}\n544e2e3b-f421-4a59-bcd6-05d57791219b@{ shape: rounded, label: \"*C S V File*\n**CSVFile:DataLake:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv**\"}\n42c5ef2f-035d-4cf8-9897-b5da133f8103==>|\"Data Flow\"|544e2e3b-f421-4a59-bcd6-05d57791219b\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"Copy Certified Data [Data Flow]\"|e11831d8-d861-4358-937a-f4f1b7be3a94\n372ae47b-5f11-42c7-91d4-2612736e60ef@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService037de6b4-bd02-4c4b-a5d8-2f53358308e1**\"}\n372ae47b-5f11-42c7-91d4-2612736e60ef==>|\"Data Flow\"|d450fa0b-c432-47cd-948a-9e546794f3a7\nf4677697-fa98-4d93-b55e-9a36b40db6bc@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService310dfaf8-fc97-408a-94be-d5490625e293**\"}\nf4677697-fa98-4d93-b55e-9a36b40db6bc==>|\"Data Flow\"|929f82e1-97e7-4c1f-b8ed-9764567c1717\nf2451a23-5e7a-4148-a97a-c63332d1c982@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService4b3efcac-2d08-430a-a6b8-fb66334d4ae1**\"}\nf2451a23-5e7a-4148-a97a-c63332d1c982==>|\"Data Flow\"|f58d9f5e-a019-4c4f-8c5b-cb173709c604\n6d7acf4e-0d61-4744-b7c3-e09de9269b70@{ shape: rounded, label: \"*Organization*\n**Organization::Oak Dene Hospital**\"}\n7f96a9b4-0270-41fb-be5b-230d3071b05d@{ shape: rounded, label: \"*Data Folder*\n**Data Folder:.landing-area/hospitals/oak-dene/clinical-trials/drop-foot**\"}\n6d7acf4e-0d61-4744-b7c3-e09de9269b70==>|\"publish weekly measurements [Data Flow]\"|7f96a9b4-0270-41fb-be5b-230d3071b05d\n55cf3959-34f6-44c6-b289-ecfe29e27592@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServiceb613a1c2-9c4a-4697-882c-2f74f9bd8586**\"}\n55cf3959-34f6-44c6-b289-ecfe29e27592==>|\"Data Flow\"|666a520c-8e33-4cd0-9147-9e4674a0d678\nf61dad34-674f-436e-8143-6a35b6343804@{ shape: rounded, label: \"*Organization*\n**Organization::Old Market Hospital**\"}\nf61dad34-674f-436e-8143-6a35b6343804==>|\"publish weekly measurements [Data Flow]\"|b2b8ade5-018d-4f15-afcd-e31c2a993335\n3b286b32-d485-449b-98d2-136b87b5889e@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_archivedOn_Wed Apr 23 20:53:02 BST 2025**\"}\n3b286b32-d485-449b-98d2-136b87b5889e==>|\"Data Flow\"|f4677697-fa98-4d93-b55e-9a36b40db6bc\nf5cf8d5d-2f6f-42b9-b5c9-0f3b94867979@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService9f506af2-9bf6-4798-8f60-838dd576af8b**\"}\nf5cf8d5d-2f6f-42b9-b5c9-0f3b94867979==>|\"Data Flow\"|28a9cb83-2fea-4f52-9c7f-890a1724c352\nc48550e0-a99c-4bdc-b98a-20a7b1cd1bc6@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week4.csv_archivedOn_Wed Apr 23 20:59:08 BST 2025**\"}\nc48550e0-a99c-4bdc-b98a-20a7b1cd1bc6==>|\"Data Flow\"|f5cf8d5d-2f6f-42b9-b5c9-0f3b94867979\n1215334f-7da2-4bee-b515-a348998a4049@{ shape: rounded, label: \"*Organization*\n**Organization::Hampton Hospital**\"}\n03f29205-02b3-4655-b487-42d0add0c3d7@{ shape: rounded, label: \"*Data Folder*\n**Data Folder:.landing-area/hospitals/hampton/clinical-trials/drop-foot**\"}\n1215334f-7da2-4bee-b515-a348998a4049==>|\"publish weekly measurements [Data Flow]\"|03f29205-02b3-4655-b487-42d0add0c3d7\n079ad4d8-d0c5-4b1b-a1e9-2897af2699c6@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_archivedOn_Wed Apr 23 20:53:03 BST 2025**\"}\n9d5713a2-296c-4f40-a68e-1f2c1565ec12@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService98114640-fdd6-418a-afe8-237c2ab7c876**\"}\n079ad4d8-d0c5-4b1b-a1e9-2897af2699c6==>|\"Data Flow\"|9d5713a2-296c-4f40-a68e-1f2c1565ec12\n9d5713a2-296c-4f40-a68e-1f2c1565ec12==>|\"Data Flow\"|e69f7e36-0a33-4559-b1a3-d864a7118814\ncaca62a1-11cd-40f9-8165-4f93e241d5ad@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_archivedOn_Wed Apr 23 20:52:41 BST 2025**\"}\ncaca62a1-11cd-40f9-8165-4f93e241d5ad==>|\"Data Flow\"|372ae47b-5f11-42c7-91d4-2612736e60ef\nd2b1a33b-6061-4524-9581-c3986df0bf01@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_archivedOn_Wed Apr 23 20:52:58 BST 2025**\"}\nd8209bc8-a9eb-4a98-8d98-8835316f77d1@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService51620f1e-3aa3-447c-90e2-795701e20d20**\"}\nd2b1a33b-6061-4524-9581-c3986df0bf01==>|\"Data Flow\"|d8209bc8-a9eb-4a98-8d98-8835316f77d1\n42e10704-22db-47d0-af6e-5dbf937a424f@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_archivedOn_Wed Apr 23 20:52:57 BST 2025**\"}\n42e10704-22db-47d0-af6e-5dbf937a424f==>|\"Data Flow\"|42c5ef2f-035d-4cf8-9897-b5da133f8103\nfba30349-2c0c-4f56-b8fa-022a263e8b95@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServicebb02d821-662b-42d5-826d-6a4790abb13c**\"}\nfba30349-2c0c-4f56-b8fa-022a263e8b95==>|\"Data Flow\"|3d2f0014-b01a-4254-8c08-5249f99c4fac\n03f29205-02b3-4655-b487-42d0add0c3d7==>|\"new file notification [Data Flow]\"|1b98cdac-dd0a-4621-93db-99ef5a1098bc\n628b1e7f-64d0-4305-8a10-aa3204ac338d==>|\"Data Flow\"|53a51fab-cb88-4cc1-916d-a378ab9aa03f\nd8209bc8-a9eb-4a98-8d98-8835316f77d1==>|\"Data Flow\"|745e39e9-04ef-4a0f-afdc-03a4c5f4448f\n752ea2fe-6550-4914-be89-e5af421b8a96@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\n226fa61e-ab4a-458e-950c-520365b68394@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService493f9ec0-d778-4437-80c6-54aed07c693e**\"}\n752ea2fe-6550-4914-be89-e5af421b8a96==>|\"Data Flow\"|226fa61e-ab4a-458e-950c-520365b68394\nfe7adc3b-b32e-4960-895c-19d75d5eab5a@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week3.csv_archivedOn_Wed Apr 23 20:59:07 BST 2025**\"}\nfe7adc3b-b32e-4960-895c-19d75d5eab5a==>|\"Data Flow\"|16299e50-a351-4ad2-8027-898ec8a3176d\n19fcbdb8-eff0-42c9-98ed-e99ee8d9e3f7==>|\"Data Flow\"|f7eca8b6-ac11-4e90-bb38-8fc0075880d5\n5f784052-0b04-4d61-b2d6-94b7eeba726a@{ shape: rounded, label: \"*Governance Action Process*\n**Coco:GovernanceActionProcess:PROJ-CT-TBDF:Oak Dene Hospital:WeeklyMeasurements:Onboarding**\"}\n5f784052-0b04-4d61-b2d6-94b7eeba726a==>|\"delivered validated file [Data Flow]\"|7e296f64-2257-44fd-8bc7-6312bb6d7af6\n9043f0f9-dad5-4cfb-ad6f-6b7f80ee8848@{ shape: rounded, label: \"*Governance Action Process*\n**Coco:GovernanceActionProcess:PROJ-CT-TBDF:Old Market Hospital:WeeklyMeasurements:Onboarding**\"}\n9043f0f9-dad5-4cfb-ad6f-6b7f80ee8848==>|\"delivered validated file [Data Flow]\"|7e296f64-2257-44fd-8bc7-6312bb6d7af6\n1b98cdac-dd0a-4621-93db-99ef5a1098bc==>|\"new file trigger [Data Flow]\"|5f784052-0b04-4d61-b2d6-94b7eeba726a\n1b98cdac-dd0a-4621-93db-99ef5a1098bc==>|\"new file trigger [Data Flow]\"|9043f0f9-dad5-4cfb-ad6f-6b7f80ee8848\n37b81f25-beea-44e2-83a6-c5d41724e026==>|\"Data Flow\"|79b79758-1eaf-4634-989f-6c1b06678402\nfeaaaa69-0af0-434d-b6ca-3e3676e4757b@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Hampton Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_archivedOn_Wed Apr 23 20:52:59 BST 2025**\"}\nfeaaaa69-0af0-434d-b6ca-3e3676e4757b==>|\"Data Flow\"|55cf3959-34f6-44c6-b289-ecfe29e27592\ne9c3bf8e-67f6-4672-8a28-a8cfb92736ca@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week2.csv_archivedOn_Wed Apr 23 20:59:06 BST 2025**\"}\n70a1a412-50bb-4a81-9b86-0003ba186c59@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServiceb05e1b8f-b2ad-4b36-887a-efa97870c988**\"}\ne9c3bf8e-67f6-4672-8a28-a8cfb92736ca==>|\"Data Flow\"|70a1a412-50bb-4a81-9b86-0003ba186c59\n226fa61e-ab4a-458e-950c-520365b68394==>|\"Data Flow\"|f92fdc60-e513-4724-8831-f198de24f477\n7f96a9b4-0270-41fb-be5b-230d3071b05d==>|\"new file notification [Data Flow]\"|1b98cdac-dd0a-4621-93db-99ef5a1098bc\n042b00c9-7614-466b-b501-f6c90ed34b51==>|\"delivered validated file [Data Flow]\"|7e296f64-2257-44fd-8bc7-6312bb6d7af6\n52fc2088-e6f3-442f-af0e-404f25d484be@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Oak Dene Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_archivedOn_Wed Apr 23 20:53:01 BST 2025**\"}\n52fc2088-e6f3-442f-af0e-404f25d484be==>|\"Data Flow\"|f2451a23-5e7a-4148-a97a-c63332d1c982\n86332826-a19d-41fb-80eb-b810fc617023@{ shape: notch-pent, label: \"*C S V File*\n**CSVFile:LandingArea:ClinicalTrial:PROJ-CT-TBDF:WeeklyMeasurement:Old Market Hospital:Wed Apr 23 20:44:43 BST 2025:/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/old-market/clinical-trials/drop-foot/week5.csv_archivedOn_Wed Apr 23 20:59:16 BST 2025**\"}\n86332826-a19d-41fb-80eb-b810fc617023==>|\"Data Flow\"|fba30349-2c0c-4f56-b8fa-022a263e8b95\n70a1a412-50bb-4a81-9b86-0003ba186c59==>|\"Data Flow\"|b25d01d9-459f-4d2a-8eb2-400109fc07e2\n3d47b8a7-e7c1-4630-8d88-e4becd388f4d@{ shape: rounded, label: \"*Data Folder*\n**Unity Catalog Volume:http:/ /localhost:8087:clinical_trials.teddy_bear_drop_foot.weekly_measurements**\"}\n7e296f64-2257-44fd-8bc7-6312bb6d7af6==>|\"get files from storage [Process Call]\"|3d47b8a7-e7c1-4630-8d88-e4becd388f4d\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f-->|\"Implemented By\"|b2b8ade5-018d-4f15-afcd-e31c2a993335\n7f5dca65-50b4-4103-9ac7-3a406a09047a-->|\"Implemented By\"|5f784052-0b04-4d61-b2d6-94b7eeba726a\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f-->|\"Implemented By\"|03f29205-02b3-4655-b487-42d0add0c3d7\n7f5dca65-50b4-4103-9ac7-3a406a09047a-->|\"Implemented By\"|9043f0f9-dad5-4cfb-ad6f-6b7f80ee8848\n26c07ca4-3b8e-484b-812b-36c1ace4b275-->|\"Implemented By\"|e11831d8-d861-4358-937a-f4f1b7be3a94\n7f5dca65-50b4-4103-9ac7-3a406a09047a-->|\"Implemented By\"|042b00c9-7614-466b-b501-f6c90ed34b51\n07705e15-efff-4f80-8992-f04ac85e0ef1-->|\"Implemented By\"|1b98cdac-dd0a-4621-93db-99ef5a1098bc\nd48f579f-76d3-4c49-b1b4-575f5645a9d0-->|\"Implemented By\"|4b6722d1-9168-4f1f-b2a4-26bc13b51512\na5d4d638-6836-47e5-99d0-fdcde637e13f-->|\"Implemented By\"|7e296f64-2257-44fd-8bc7-6312bb6d7af6\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f-->|\"Implemented By\"|7f96a9b4-0270-41fb-be5b-230d3071b05d\nend\nend\nsubgraph Overview\n6859e374-8880-473a-b90e-d9f55935d5dd@{ shape: flip-tri, label: \"*Information Supply Chain*\n**Clinical Trial Treatment Validation:PROJ-CT-TBDF**\"}\n849df855-d1a0-4657-bf8c-173348863b3e@{ shape: text, label: \"*Description*\n**Delivering data relating to the Teddy Bear Drop Foot Clinical Trial clinical trial from the hospitals to the Coco Researchers so that they can then determine the efficacy of the treatment to report to the regulators.**\"}\n1d6a94e5-bae0-4c39-900b-af8730a11d3d@{ shape: text, label: \"*Purpose*\n**Deliver patient measurement data from hospitals to data scientists in research.**\"}\n88ea96ad-000b-470a-b40f-77677d816d61@{ shape: text, label: \"*Purpose*\n**Ensure incoming data is only from certified hospitals.**\"}\n2f598a50-1841-4a97-9216-e9a4ec1ee190@{ shape: text, label: \"*Purpose*\n**Ensure incoming data is validated and catalogued.**\"}\n6d072f95-53bd-4f40-94cc-cf63b762e50d@{ shape: text, label: \"*Purpose*\n**Ensure data and process owners are informed of key milestones and issues requiring attention.**\"}\n4b025202-7424-4fdf-a0e5-0137f02724b1@{ shape: text, label: \"*Purpose*\n**Ensure the process of data capture and treatment validation is transparent and auditable.**\"}\nee6ef0c2-855a-496d-adc7-70e46751af30@{ shape: text, label: \"*Purpose*\n**Ensure the treatment validation report is complete and regulatory compliant.**\"}\nend\nOverview~~~Details\n9032c840-ea1e-464c-9050-d95669d96f92-,|\"Anchors\"|e9c3bf8e-67f6-4672-8a28-a8cfb92736ca\ne19b2f8c-2513-4689-befd-0fdfddacea3f-,|\"Anchors\"|e9c3bf8e-67f6-4672-8a28-a8cfb92736ca\n978ee626-c755-4e99-ac94-a1d295fc8f68-,|\"Anchors\"|e9c3bf8e-67f6-4672-8a28-a8cfb92736ca\n85bf997e-0548-43c9-bdc7-20eaf2e82878-,|\"Anchors\"|e9c3bf8e-67f6-4672-8a28-a8cfb92736ca\n64bfac1b-c502-454a-92cd-f410d643dfd8-,|\"Anchors\"|79b79758-1eaf-4634-989f-6c1b06678402\ne8ac318b-885c-4877-a353-c7b7c63f73b1-,|\"Anchors\"|feaaaa69-0af0-434d-b6ca-3e3676e4757b\nc192ad79-3fd5-4e07-a2d0-276dc2050481-,|\"Anchors\"|feaaaa69-0af0-434d-b6ca-3e3676e4757b\n479728fe-c9b6-4349-a4c8-8e87b2190642-,|\"Anchors\"|feaaaa69-0af0-434d-b6ca-3e3676e4757b\n3d2d1b40-fb0d-42ee-82bc-80466ac6437d-,|\"Anchors\"|feaaaa69-0af0-434d-b6ca-3e3676e4757b\n967c57c1-d754-4de8-8f28-562aac57528f@{ shape: rounded, label: \"*Deployed Database Schema*\n**967c57c1-d754-4de8-8f28-562aac57528f**\"}\n3d47b8a7-e7c1-4630-8d88-e4becd388f4d-,|\"Anchors\"|967c57c1-d754-4de8-8f28-562aac57528f\n2a3b2133-7705-4dd3-96f4-59e68c0c584a-,|\"Anchors\"|3b286b32-d485-449b-98d2-136b87b5889e\n93e81b66-bd51-408b-9dab-4ffc61cda040-,|\"Anchors\"|3b286b32-d485-449b-98d2-136b87b5889e\nb3773e8d-d562-434d-a455-a567723c48ac-,|\"Anchors\"|3b286b32-d485-449b-98d2-136b87b5889e\n948df020-b9f1-472c-92d8-44699be0022e-,|\"Anchors\"|3b286b32-d485-449b-98d2-136b87b5889e\ncd5ada12-c854-4818-9bf9-e48584a041b8-,|\"Anchors\"|079ad4d8-d0c5-4b1b-a1e9-2897af2699c6\ne07be6ba-3f54-406a-bd01-dcf994a3000e-,|\"Anchors\"|079ad4d8-d0c5-4b1b-a1e9-2897af2699c6\n5b0cf874-3f00-4440-9d3b-15a3a88ed84d-,|\"Anchors\"|079ad4d8-d0c5-4b1b-a1e9-2897af2699c6\nc93b3371-ec01-41c5-b912-339a04f10de1-,|\"Anchors\"|079ad4d8-d0c5-4b1b-a1e9-2897af2699c6\na1065c6a-e8dc-48be-a0ce-daab251b8279-,|\"Anchors\"|3d2f0014-b01a-4254-8c08-5249f99c4fac\n82f10cb5-7625-4fdd-a72a-f2a388ab871d-,|\"Anchors\"|53a51fab-cb88-4cc1-916d-a378ab9aa03f\n9f6a9170-5e20-4e37-82c8-80f21d901181-,|\"Anchors\"|28a9cb83-2fea-4f52-9c7f-890a1724c352\naa16f792-e97e-4bd0-abfb-029059d4c8a9-,|\"Anchors\"|752ea2fe-6550-4914-be89-e5af421b8a96\n6feeace9-5620-42e6-b36e-5916ee7e8b41-,|\"Anchors\"|752ea2fe-6550-4914-be89-e5af421b8a96\nce5d56af-7475-4030-9d88-d4ed7a485c38-,|\"Anchors\"|752ea2fe-6550-4914-be89-e5af421b8a96\n68d912f0-9c3c-469b-8f45-9f8f6012b18f-,|\"Anchors\"|752ea2fe-6550-4914-be89-e5af421b8a96\n47c3142d-2e72-4d97-b495-324ff303bb1c-,|\"Anchors\"|666a520c-8e33-4cd0-9147-9e4674a0d678\n5d6faf77-bca0-473a-9da4-1f8f5e203f75-,|\"Anchors\"|86332826-a19d-41fb-80eb-b810fc617023\ne96d69ea-5a07-4d77-9862-19a53ac71bf8-,|\"Anchors\"|86332826-a19d-41fb-80eb-b810fc617023\n7ba8acda-5c6b-4f48-a381-a0e14382a08d-,|\"Anchors\"|86332826-a19d-41fb-80eb-b810fc617023\n9d5ee104-38b6-435c-96b4-37183cca58f8-,|\"Anchors\"|86332826-a19d-41fb-80eb-b810fc617023\n6f9f66fd-2e41-4b7d-825d-124288cad6a4-,|\"Anchors\"|42e10704-22db-47d0-af6e-5dbf937a424f\n4d2582e0-3ad2-4c70-add5-ce078b0fd3b8-,|\"Anchors\"|42e10704-22db-47d0-af6e-5dbf937a424f\nc7f82794-9ebd-4ba1-b29f-1b3b224551a7-,|\"Anchors\"|42e10704-22db-47d0-af6e-5dbf937a424f\n1ce9095b-7fe6-410f-ac0e-78e3cac5814a-,|\"Anchors\"|42e10704-22db-47d0-af6e-5dbf937a424f\nea9cc426-f108-4365-84be-0e2f95b52b2c-,|\"Anchors\"|1b34b404-180d-4e35-bec0-c43b563425fe\neb20ddb9-32fa-47c4-883e-e737d4e25320-,|\"Anchors\"|b25d01d9-459f-4d2a-8eb2-400109fc07e2\nad82f669-a81d-440f-9a6f-586e4107266d-,|\"Anchors\"|e69f7e36-0a33-4559-b1a3-d864a7118814\n4a988bae-30cb-4cac-a375-d8e8aa25b893-,|\"Anchors\"|745e39e9-04ef-4a0f-afdc-03a4c5f4448f\n87876c24-f38e-4f0f-87fa-08b59e2ace15-,|\"Anchors\"|929f82e1-97e7-4c1f-b8ed-9764567c1717\n312d3284-8962-4ea2-9c20-3ce1eb9b9d88-,|\"Anchors\"|d2b1a33b-6061-4524-9581-c3986df0bf01\nb0815266-f62c-4491-b516-3bca365bbb62-,|\"Anchors\"|d2b1a33b-6061-4524-9581-c3986df0bf01\n3e87d13b-2d86-48df-8073-3b24b945dd80-,|\"Anchors\"|d2b1a33b-6061-4524-9581-c3986df0bf01\n697d48ea-459c-4a16-840e-ecec8dfc2ca9-,|\"Anchors\"|d2b1a33b-6061-4524-9581-c3986df0bf01\nab4a83f8-924f-4696-aa62-a74f65f3fe17-,|\"Anchors\"|f5626f07-5a4f-4ad6-801e-c66f9415a885\nbe171af3-db8d-41f2-91ab-db1edf0aad0d-,|\"Anchors\"|f5626f07-5a4f-4ad6-801e-c66f9415a885\n9cea142e-2a6f-4e2e-8a4a-5148490e7de1-,|\"Anchors\"|f5626f07-5a4f-4ad6-801e-c66f9415a885\n8f9d3489-cca9-442a-bbe7-1486560fdfb7-,|\"Anchors\"|f5626f07-5a4f-4ad6-801e-c66f9415a885\n5f28ccec-d4fc-41cd-826e-90fa203953c9-,|\"Anchors\"|f7eca8b6-ac11-4e90-bb38-8fc0075880d5\ncfb00a09-801a-44fc-a29f-5e725919bee6-,|\"Anchors\"|52fc2088-e6f3-442f-af0e-404f25d484be\nb418a842-3907-40d1-880d-f4bdc465a285-,|\"Anchors\"|52fc2088-e6f3-442f-af0e-404f25d484be\nb37b8106-c884-494d-9aed-10e1df027e2e-,|\"Anchors\"|52fc2088-e6f3-442f-af0e-404f25d484be\n4ce00c3c-c037-4b80-a56d-bb25ca40619e-,|\"Anchors\"|52fc2088-e6f3-442f-af0e-404f25d484be\n3d642257-8ef7-4ef1-822a-855e4fca1162-,|\"Anchors\"|caca62a1-11cd-40f9-8165-4f93e241d5ad\na4fcebd2-f475-40f9-839e-17a1bbe879c8-,|\"Anchors\"|caca62a1-11cd-40f9-8165-4f93e241d5ad\n3dadf467-46a3-4b9a-aad9-87ba79418715-,|\"Anchors\"|caca62a1-11cd-40f9-8165-4f93e241d5ad\n9c5ad26e-0cfc-4f32-a64a-f4e9f0cc8526-,|\"Anchors\"|caca62a1-11cd-40f9-8165-4f93e241d5ad\nc26a1f9f-2686-4697-b1d9-28f6dd416149-,|\"Anchors\"|d450fa0b-c432-47cd-948a-9e546794f3a7\n750b9fb3-aae2-4d14-b500-e136dfd48d1e-,|\"Anchors\"|f58d9f5e-a019-4c4f-8c5b-cb173709c604\n2480ea7f-c268-435c-a475-06428798effd-,|\"Anchors\"|c48550e0-a99c-4bdc-b98a-20a7b1cd1bc6\n16a76505-cca4-4af7-8ac5-8143ab3f65d4-,|\"Anchors\"|c48550e0-a99c-4bdc-b98a-20a7b1cd1bc6\ndcad51d6-66da-49ef-b48e-0db24b27a079-,|\"Anchors\"|c48550e0-a99c-4bdc-b98a-20a7b1cd1bc6\n91e5f6e9-d836-44bf-a88a-64df4e30f48d-,|\"Anchors\"|c48550e0-a99c-4bdc-b98a-20a7b1cd1bc6\n6c52b8a5-11b6-495c-b3b8-98b84c37bfaa@{ shape: rounded, label: \"*Data Folder*\n**6c52b8a5-11b6-495c-b3b8-98b84c37bfaa**\"}\n79b79758-1eaf-4634-989f-6c1b06678402-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\nf7eca8b6-ac11-4e90-bb38-8fc0075880d5-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n3d2f0014-b01a-4254-8c08-5249f99c4fac-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n53a51fab-cb88-4cc1-916d-a378ab9aa03f-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n28a9cb83-2fea-4f52-9c7f-890a1724c352-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n666a520c-8e33-4cd0-9147-9e4674a0d678-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\nf58d9f5e-a019-4c4f-8c5b-cb173709c604-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\nd450fa0b-c432-47cd-948a-9e546794f3a7-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n1b34b404-180d-4e35-bec0-c43b563425fe-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\nb25d01d9-459f-4d2a-8eb2-400109fc07e2-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\ne69f7e36-0a33-4559-b1a3-d864a7118814-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n544e2e3b-f421-4a59-bcd6-05d57791219b-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n745e39e9-04ef-4a0f-afdc-03a4c5f4448f-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n929f82e1-97e7-4c1f-b8ed-9764567c1717-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\nf92fdc60-e513-4724-8831-f198de24f477-,|\"Anchors\"|6c52b8a5-11b6-495c-b3b8-98b84c37bfaa\n3677d8ff-d3ec-4a23-9296-1b026daa80bb-,|\"Anchors\"|0a9c3d3d-8f37-4487-aeae-d20bee32d16a\nb9757c30-75a3-4460-b4c0-e9423770dfd1-,|\"Anchors\"|0a9c3d3d-8f37-4487-aeae-d20bee32d16a\nc7b3da12-4d8f-4a61-b182-10e7ba92bac1-,|\"Anchors\"|0a9c3d3d-8f37-4487-aeae-d20bee32d16a\nbc21e09c-7664-4555-b481-37e0b9ddd665-,|\"Anchors\"|0a9c3d3d-8f37-4487-aeae-d20bee32d16a\n72cf017b-1fa5-4bb0-94a6-1d59f81af268@{ shape: rounded, label: \"*Deployed Connector*\n**72cf017b-1fa5-4bb0-94a6-1d59f81af268**\"}\n226fa61e-ab4a-458e-950c-520365b68394-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n16299e50-a351-4ad2-8027-898ec8a3176d-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\nf4677697-fa98-4d93-b55e-9a36b40db6bc-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n42c5ef2f-035d-4cf8-9897-b5da133f8103-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n37b81f25-beea-44e2-83a6-c5d41724e026-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n628b1e7f-64d0-4305-8a10-aa3204ac338d-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\nf2451a23-5e7a-4148-a97a-c63332d1c982-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\nfba30349-2c0c-4f56-b8fa-022a263e8b95-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n55cf3959-34f6-44c6-b289-ecfe29e27592-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\nd8209bc8-a9eb-4a98-8d98-8835316f77d1-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n70a1a412-50bb-4a81-9b86-0003ba186c59-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n9d5713a2-296c-4f40-a68e-1f2c1565ec12-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\nf5cf8d5d-2f6f-42b9-b5c9-0f3b94867979-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n19fcbdb8-eff0-42c9-98ed-e99ee8d9e3f7-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n372ae47b-5f11-42c7-91d4-2612736e60ef-,|\"Anchors\"|72cf017b-1fa5-4bb0-94a6-1d59f81af268\n1c1de4b7-4c19-4f88-a40c-ea650eba3629-,|\"Anchors\"|fe7adc3b-b32e-4960-895c-19d75d5eab5a\nd367d7cb-bc7f-4572-b883-895052f07935-,|\"Anchors\"|fe7adc3b-b32e-4960-895c-19d75d5eab5a\n6c854386-b00e-4a2d-906b-9e3d937cd20c-,|\"Anchors\"|fe7adc3b-b32e-4960-895c-19d75d5eab5a\n4648bd1a-a9dc-4ee1-98e0-7e6534e3c929-,|\"Anchors\"|fe7adc3b-b32e-4960-895c-19d75d5eab5a\n7af2fa96-ab8e-4e0f-beef-3b3406a23031-,|\"Anchors\"|6703dd61-76fe-4741-b55a-967a4eb2bda3\n985104b8-bf32-41e2-91fa-0b354f8861d8-,|\"Anchors\"|6703dd61-76fe-4741-b55a-967a4eb2bda3\ne30903c5-d863-4ea8-941f-9dde8c0d1472-,|\"Anchors\"|6703dd61-76fe-4741-b55a-967a4eb2bda3\nba037aed-a61e-4bfc-91e7-de95f7fa3e2f-,|\"Anchors\"|6703dd61-76fe-4741-b55a-967a4eb2bda3\n7de8760e-5630-4f74-b92d-6534d37713a8-,|\"Anchors\"|544e2e3b-f421-4a59-bcd6-05d57791219b\na0688d35-64e0-4b46-b9b7-a91b6977c5b2-,|\"Anchors\"|f92fdc60-e513-4724-8831-f198de24f477\nstyle 48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 2a3b2133-7705-4dd3-96f4-59e68c0c584a color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 9043f0f9-dad5-4cfb-ad6f-6b7f80ee8848 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 9f6a9170-5e20-4e37-82c8-80f21d901181 color:#000000, fill:#e0ab18, stroke:#004563\nstyle f61dad34-674f-436e-8143-6a35b6343804 color:#000000, fill:#e0ab18, stroke:#004563\nstyle e9c3bf8e-67f6-4672-8a28-a8cfb92736ca color:#004563, fill:#f8f8ff, stroke:#004563\nstyle c7b3da12-4d8f-4a61-b182-10e7ba92bac1 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle b418a842-3907-40d1-880d-f4bdc465a285 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 79b79758-1eaf-4634-989f-6c1b06678402 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 3e87d13b-2d86-48df-8073-3b24b945dd80 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle Segments color:#004563, fill:#FFFFFF, stroke:#004563\nstyle e96d69ea-5a07-4d77-9862-19a53ac71bf8 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 37b81f25-beea-44e2-83a6-c5d41724e026 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 752ea2fe-6550-4914-be89-e5af421b8a96 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 849df855-d1a0-4657-bf8c-173348863b3e color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle a5d4d638-6836-47e5-99d0-fdcde637e13f color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle Design color:#000000, fill:#b0c4de, stroke:#000000\nstyle 38635d38-f728-400d-a8ec-7c26e68b7c0f color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 93e81b66-bd51-408b-9dab-4ffc61cda040 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle b25d01d9-459f-4d2a-8eb2-400109fc07e2 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 042b00c9-7614-466b-b501-f6c90ed34b51 color:#000000, fill:#e0ab18, stroke:#004563\nstyle f5cf8d5d-2f6f-42b9-b5c9-0f3b94867979 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 9c5ad26e-0cfc-4f32-a64a-f4e9f0cc8526 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle eb20ddb9-32fa-47c4-883e-e737d4e25320 color:#000000, fill:#e0ab18, stroke:#004563\nstyle cfb00a09-801a-44fc-a29f-5e725919bee6 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 5f28ccec-d4fc-41cd-826e-90fa203953c9 color:#000000, fill:#e0ab18, stroke:#004563\nstyle a4fcebd2-f475-40f9-839e-17a1bbe879c8 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle f4677697-fa98-4d93-b55e-9a36b40db6bc color:#000000, fill:#e0ab18, stroke:#004563\nstyle 42c5ef2f-035d-4cf8-9897-b5da133f8103 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 6c854386-b00e-4a2d-906b-9e3d937cd20c color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 628b1e7f-64d0-4305-8a10-aa3204ac338d color:#000000, fill:#e0ab18, stroke:#004563\nstyle b37b8106-c884-494d-9aed-10e1df027e2e color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 03f29205-02b3-4655-b487-42d0add0c3d7 color:#000000, fill:#e0ab18, stroke:#004563\nstyle caca62a1-11cd-40f9-8165-4f93e241d5ad color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 5f784052-0b04-4d61-b2d6-94b7eeba726a color:#000000, fill:#e0ab18, stroke:#004563\nstyle Details color:#FFFFFF, fill:#FFFFFF, stroke:#FFFFFF\nstyle e19b2f8c-2513-4689-befd-0fdfddacea3f color:#004563, fill:#f8f8ff, stroke:#004563\nstyle c48550e0-a99c-4bdc-b98a-20a7b1cd1bc6 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 6d072f95-53bd-4f40-94cc-cf63b762e50d color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 70a1a412-50bb-4a81-9b86-0003ba186c59 color:#000000, fill:#e0ab18, stroke:#004563\nstyle ad82f669-a81d-440f-9a6f-586e4107266d color:#000000, fill:#e0ab18, stroke:#004563\nstyle 4b025202-7424-4fdf-a0e5-0137f02724b1 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 87876c24-f38e-4f0f-87fa-08b59e2ace15 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 312d3284-8962-4ea2-9c20-3ce1eb9b9d88 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle b0815266-f62c-4491-b516-3bca365bbb62 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 967c57c1-d754-4de8-8f28-562aac57528f color:#000000, fill:#FFFFCC, stroke:#004563\nstyle 3d2f0014-b01a-4254-8c08-5249f99c4fac color:#000000, fill:#e0ab18, stroke:#004563\nstyle Implementation color:#004563, fill:#b7c0c7, stroke:#004563\nstyle a1065c6a-e8dc-48be-a0ce-daab251b8279 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 4ce00c3c-c037-4b80-a56d-bb25ca40619e color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 42e10704-22db-47d0-af6e-5dbf937a424f color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 1c1de4b7-4c19-4f88-a40c-ea650eba3629 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle cd5ada12-c854-4818-9bf9-e48584a041b8 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 7edca02c-e726-4570-815c-280bdf5498b9 color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 0bf2547c-937c-41b6-814f-6284849271a1 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle fba30349-2c0c-4f56-b8fa-022a263e8b95 color:#000000, fill:#e0ab18, stroke:#004563\nstyle dcad51d6-66da-49ef-b48e-0db24b27a079 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 19fcbdb8-eff0-42c9-98ed-e99ee8d9e3f7 color:#000000, fill:#e0ab18, stroke:#004563\nstyle d2b1a33b-6061-4524-9581-c3986df0bf01 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle f7eca8b6-ac11-4e90-bb38-8fc0075880d5 color:#000000, fill:#e0ab18, stroke:#004563\nstyle f5626f07-5a4f-4ad6-801e-c66f9415a885 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle bc21e09c-7664-4555-b481-37e0b9ddd665 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 948df020-b9f1-472c-92d8-44699be0022e color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 4648bd1a-a9dc-4ee1-98e0-7e6534e3c929 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle e8ac318b-885c-4877-a353-c7b7c63f73b1 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle f58d9f5e-a019-4c4f-8c5b-cb173709c604 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 4d2582e0-3ad2-4c70-add5-ce078b0fd3b8 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle d8209bc8-a9eb-4a98-8d98-8835316f77d1 color:#000000, fill:#e0ab18, stroke:#004563\nstyle b3773e8d-d562-434d-a455-a567723c48ac color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 64bfac1b-c502-454a-92cd-f410d643dfd8 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 0a9c3d3d-8f37-4487-aeae-d20bee32d16a color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 72cf017b-1fa5-4bb0-94a6-1d59f81af268 color:#000000, fill:#FFFFCC, stroke:#004563\nstyle 544e2e3b-f421-4a59-bcd6-05d57791219b color:#000000, fill:#e0ab18, stroke:#004563\nstyle 1ce9095b-7fe6-410f-ac0e-78e3cac5814a color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 3677d8ff-d3ec-4a23-9296-1b026daa80bb color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 6f9f66fd-2e41-4b7d-825d-124288cad6a4 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 226fa61e-ab4a-458e-950c-520365b68394 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 985104b8-bf32-41e2-91fa-0b354f8861d8 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle ba037aed-a61e-4bfc-91e7-de95f7fa3e2f color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 3dadf467-46a3-4b9a-aad9-87ba79418715 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle a0688d35-64e0-4b46-b9b7-a91b6977c5b2 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7ba8acda-5c6b-4f48-a381-a0e14382a08d color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 28a9cb83-2fea-4f52-9c7f-890a1724c352 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 2f598a50-1841-4a97-9216-e9a4ec1ee190 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 666a520c-8e33-4cd0-9147-9e4674a0d678 color:#000000, fill:#e0ab18, stroke:#004563\nstyle e11831d8-d861-4358-937a-f4f1b7be3a94 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 1b34b404-180d-4e35-bec0-c43b563425fe color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7f96a9b4-0270-41fb-be5b-230d3071b05d color:#000000, fill:#e0ab18, stroke:#004563\nstyle 91e5f6e9-d836-44bf-a88a-64df4e30f48d color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 9d5ee104-38b6-435c-96b4-37183cca58f8 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 1c150d6e-30cf-481c-9afb-3b06c9c9e78f color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 745e39e9-04ef-4a0f-afdc-03a4c5f4448f color:#000000, fill:#e0ab18, stroke:#004563\nstyle 929f82e1-97e7-4c1f-b8ed-9764567c1717 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 07705e15-efff-4f80-8992-f04ac85e0ef1 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 372ae47b-5f11-42c7-91d4-2612736e60ef color:#000000, fill:#e0ab18, stroke:#004563\nstyle 2480ea7f-c268-435c-a475-06428798effd color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 1d6a94e5-bae0-4c39-900b-af8730a11d3d color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 5d6faf77-bca0-473a-9da4-1f8f5e203f75 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 4fc47e60-f1b5-469c-b666-e1f4570e749f color:#004563, fill:#b7c0c7, stroke:#004563\nstyle d367d7cb-bc7f-4572-b883-895052f07935 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 697d48ea-459c-4a16-840e-ecec8dfc2ca9 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 6d7acf4e-0d61-4744-b7c3-e09de9269b70 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 9cea142e-2a6f-4e2e-8a4a-5148490e7de1 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle ee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 6859e374-8880-473a-b90e-d9f55935d5dd color:#FFFFFF, fill:#004563, stroke:#b7c0c7\nstyle e07be6ba-3f54-406a-bd01-dcf994a3000e color:#004563, fill:#f8f8ff, stroke:#004563\nstyle aa16f792-e97e-4bd0-abfb-029059d4c8a9 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 5b0cf874-3f00-4440-9d3b-15a3a88ed84d color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 3d642257-8ef7-4ef1-822a-855e4fca1162 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle d48f579f-76d3-4c49-b1b4-575f5645a9d0 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 3d47b8a7-e7c1-4630-8d88-e4becd388f4d color:#000000, fill:#e0ab18, stroke:#004563\nstyle 9d5713a2-296c-4f40-a68e-1f2c1565ec12 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 82f10cb5-7625-4fdd-a72a-f2a388ab871d color:#000000, fill:#e0ab18, stroke:#004563\nstyle fe7adc3b-b32e-4960-895c-19d75d5eab5a color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 04ae768e-3816-47bc-bddb-c9ae25018684 color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 68d912f0-9c3c-469b-8f45-9f8f6012b18f color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 1b98cdac-dd0a-4621-93db-99ef5a1098bc color:#000000, fill:#e0ab18, stroke:#004563\nstyle 750b9fb3-aae2-4d14-b500-e136dfd48d1e color:#000000, fill:#e0ab18, stroke:#004563\nstyle f92fdc60-e513-4724-8831-f198de24f477 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7de8760e-5630-4f74-b92d-6534d37713a8 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 16a76505-cca4-4af7-8ac5-8143ab3f65d4 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle e4303326-e418-4f77-b8e7-fd5d34717594 color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 978ee626-c755-4e99-ac94-a1d295fc8f68 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle ce5d56af-7475-4030-9d88-d4ed7a485c38 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle feaaaa69-0af0-434d-b6ca-3e3676e4757b color:#004563, fill:#f8f8ff, stroke:#004563\nstyle c192ad79-3fd5-4e07-a2d0-276dc2050481 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 3b286b32-d485-449b-98d2-136b87b5889e color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 079ad4d8-d0c5-4b1b-a1e9-2897af2699c6 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle ea9cc426-f108-4365-84be-0e2f95b52b2c color:#000000, fill:#e0ab18, stroke:#004563\nstyle 53a51fab-cb88-4cc1-916d-a378ab9aa03f color:#000000, fill:#e0ab18, stroke:#004563\nstyle 3d2d1b40-fb0d-42ee-82bc-80466ac6437d color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 86332826-a19d-41fb-80eb-b810fc617023 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 8f9d3489-cca9-442a-bbe7-1486560fdfb7 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 7af2fa96-ab8e-4e0f-beef-3b3406a23031 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 9032c840-ea1e-464c-9050-d95669d96f92 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle ab4a83f8-924f-4696-aa62-a74f65f3fe17 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 6feeace9-5620-42e6-b36e-5916ee7e8b41 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 55cf3959-34f6-44c6-b289-ecfe29e27592 color:#000000, fill:#e0ab18, stroke:#004563\nstyle b2b8ade5-018d-4f15-afcd-e31c2a993335 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 88ea96ad-000b-470a-b40f-77677d816d61 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle c7f82794-9ebd-4ba1-b29f-1b3b224551a7 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 85bf997e-0548-43c9-bdc7-20eaf2e82878 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 7e296f64-2257-44fd-8bc7-6312bb6d7af6 color:#000000, fill:#e0ab18, stroke:#004563\nstyle e69f7e36-0a33-4559-b1a3-d864a7118814 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 26c07ca4-3b8e-484b-812b-36c1ace4b275 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 16299e50-a351-4ad2-8027-898ec8a3176d color:#000000, fill:#e0ab18, stroke:#004563\nstyle 1215334f-7da2-4bee-b515-a348998a4049 color:#000000, fill:#e0ab18, stroke:#004563\nstyle e30903c5-d863-4ea8-941f-9dde8c0d1472 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle Overview color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle c26a1f9f-2686-4697-b1d9-28f6dd416149 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7f5dca65-50b4-4103-9ac7-3a406a09047a color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 52fc2088-e6f3-442f-af0e-404f25d484be color:#004563, fill:#f8f8ff, stroke:#004563\nstyle ee6ef0c2-855a-496d-adc7-70e46751af30 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 72a86eec-9734-4bc0-babb-4fec0aa7c9ff color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle d450fa0b-c432-47cd-948a-9e546794f3a7 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 4b6722d1-9168-4f1f-b2a4-26bc13b51512 color:#000000, fill:#e0ab18, stroke:#004563\nstyle b5c8da4c-f925-4cf1-8294-e43cd2c1a584 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle f2451a23-5e7a-4148-a97a-c63332d1c982 color:#000000, fill:#e0ab18, stroke:#004563\nstyle b9757c30-75a3-4460-b4c0-e9423770dfd1 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 47c3142d-2e72-4d97-b495-324ff303bb1c color:#000000, fill:#e0ab18, stroke:#004563\nstyle 6c52b8a5-11b6-495c-b3b8-98b84c37bfaa color:#000000, fill:#FFFFCC, stroke:#004563\nstyle 4a988bae-30cb-4cac-a375-d8e8aa25b893 color:#000000, fill:#e0ab18, stroke:#004563\nstyle be171af3-db8d-41f2-91ab-db1edf0aad0d color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 6703dd61-76fe-4741-b55a-967a4eb2bda3 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 479728fe-c9b6-4349-a4c8-8e87b2190642 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle c93b3371-ec01-41c5-b912-339a04f10de1 color:#004563, fill:#f8f8ff, stroke:#004563\n"
print(mermaid_graph)

render_mermaid(mermaid_graph)


In [ ]:

mermaid_graph="---\ntitle: Information Supply Chain - Clinical Trial Treatment Validation:PROJ-CT-TBDF [a2caa20a-2cb1-4d35-a96a-6493bb9036d3]\n---\nflowchart TD\n%%{init: {\"flowchart\": {\"htmlLabels\": false}} }%%\n\nsubgraph Details\nsubgraph Segments\n38635d38-f728-400d-a8ec-7c26e68b7c0f@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Weekly Measurements Onboarding**\"}\n38635d38-f728-400d-a8ec-7c26e68b7c0f==>|\"capture and catalog,\n[Information Supply Chain Link]\"|7edca02c-e726-4570-815c-280bdf5498b9\ne4303326-e418-4f77-b8e7-fd5d34717594@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Assess Treatment under Trial**\"}\ne4303326-e418-4f77-b8e7-fd5d34717594==>|\"publish results,\n[Information Supply Chain Link]\"|4fc47e60-f1b5-469c-b666-e1f4570e749f\n7edca02c-e726-4570-815c-280bdf5498b9@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Data Lake to Sandbox**\"}\n7edca02c-e726-4570-815c-280bdf5498b9==>|\"assemble,\n[Information Supply Chain Link]\"|e4303326-e418-4f77-b8e7-fd5d34717594\n04ae768e-3816-47bc-bddb-c9ae25018684@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Hospital Delivers Patient Weekly Readings**\"}\n04ae768e-3816-47bc-bddb-c9ae25018684==>|\"publish,\n[Information Supply Chain Link]\"|38635d38-f728-400d-a8ec-7c26e68b7c0f\n4fc47e60-f1b5-469c-b666-e1f4570e749f@{ shape: hex, label: \"*Information Supply Chain Segment*\n**Deliver Treatment Assessment Report**\"}\nend\nsubgraph Design\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Hospital Landing Area Folder::V1.0**\"}\n7f5dca65-50b4-4103-9ac7-3a406a09047a@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Weekly Measurements Onboarding Pipeline::V1.0**\"}\n07705e15-efff-4f80-8992-f04ac85e0ef1@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Landing Folder Cataloguer::V1.0**\"}\na5d4d638-6836-47e5-99d0-fdcde637e13f@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Weekly Measurements Data Lake Folder::V1.0**\"}\n48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Treatment Efficacy Evidence::V1.0**\"}\nb5c8da4c-f925-4cf1-8294-e43cd2c1a584@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Analyse Patient Data::V1.0**\"}\nd48f579f-76d3-4c49-b1b4-575f5645a9d0@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Treatment Validation Sandbox::V1.0**\"}\n26c07ca4-3b8e-484b-812b-36c1ace4b275@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Populate Sandbox::V1.0**\"}\nee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Hospital Processes::V1.0**\"}\n72a86eec-9734-4bc0-babb-4fec0aa7c9ff@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Assemble Treatment Assessment Report::V1.0**\"}\n0bf2547c-937c-41b6-814f-6284849271a1@{ shape: rect, label: \"*Solution Component*\n**SolutionComponent::Treatment Assessment Report Validation and Delivery::V1.0**\"}\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|1c150d6e-30cf-481c-9afb-3b06c9c9e78f\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|7f5dca65-50b4-4103-9ac7-3a406a09047a\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|07705e15-efff-4f80-8992-f04ac85e0ef1\n38635d38-f728-400d-a8ec-7c26e68b7c0f-->|\"Supporting Supply Chain,\n[Implemented By]\"|a5d4d638-6836-47e5-99d0-fdcde637e13f\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f==>|\"detect new files,\n[Solution Linking Wire]\"|07705e15-efff-4f80-8992-f04ac85e0ef1\nee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec==>|\"publish,\n[Solution Linking Wire]\"|1c150d6e-30cf-481c-9afb-3b06c9c9e78f\n07705e15-efff-4f80-8992-f04ac85e0ef1==>|\"request onboarding,\n[Solution Linking Wire]\"|7f5dca65-50b4-4103-9ac7-3a406a09047a\n7f5dca65-50b4-4103-9ac7-3a406a09047a==>|\"save new files,\n[Solution Linking Wire]\"|a5d4d638-6836-47e5-99d0-fdcde637e13f\na5d4d638-6836-47e5-99d0-fdcde637e13f==>|\"read certified files,\n[Solution Linking Wire]\"|26c07ca4-3b8e-484b-812b-36c1ace4b275\ne4303326-e418-4f77-b8e7-fd5d34717594-->|\"Supporting Supply Chain,\n[Implemented By]\"|48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5\ne4303326-e418-4f77-b8e7-fd5d34717594-->|\"Supporting Supply Chain,\n[Implemented By]\"|b5c8da4c-f925-4cf1-8294-e43cd2c1a584\ne4303326-e418-4f77-b8e7-fd5d34717594-->|\"Supporting Supply Chain,\n[Implemented By]\"|d48f579f-76d3-4c49-b1b4-575f5645a9d0\n48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5==>|\"retrieve evidence,\n[Solution Linking Wire]\"|72a86eec-9734-4bc0-babb-4fec0aa7c9ff\nb5c8da4c-f925-4cf1-8294-e43cd2c1a584==>|\"publish results,\n[Solution Linking Wire]\"|48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5\nd48f579f-76d3-4c49-b1b4-575f5645a9d0==>|\"retrieve patient data,\n[Solution Linking Wire]\"|b5c8da4c-f925-4cf1-8294-e43cd2c1a584\n26c07ca4-3b8e-484b-812b-36c1ace4b275==>|\"write patient measurements,\n[Solution Linking Wire]\"|d48f579f-76d3-4c49-b1b4-575f5645a9d0\n7edca02c-e726-4570-815c-280bdf5498b9-->|\"Supporting Supply Chain,\n[Implemented By]\"|a5d4d638-6836-47e5-99d0-fdcde637e13f\n7edca02c-e726-4570-815c-280bdf5498b9-->|\"Supporting Supply Chain,\n[Implemented By]\"|26c07ca4-3b8e-484b-812b-36c1ace4b275\n7edca02c-e726-4570-815c-280bdf5498b9-->|\"Supporting Supply Chain,\n[Implemented By]\"|d48f579f-76d3-4c49-b1b4-575f5645a9d0\n04ae768e-3816-47bc-bddb-c9ae25018684-->|\"Supporting Supply Chain,\n[Implemented By]\"|1c150d6e-30cf-481c-9afb-3b06c9c9e78f\n04ae768e-3816-47bc-bddb-c9ae25018684-->|\"Supporting Supply Chain,\n[Implemented By]\"|ee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec\n4fc47e60-f1b5-469c-b666-e1f4570e749f-->|\"Supporting Supply Chain,\n[Implemented By]\"|72a86eec-9734-4bc0-babb-4fec0aa7c9ff\n4fc47e60-f1b5-469c-b666-e1f4570e749f-->|\"Supporting Supply Chain,\n[Implemented By]\"|48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5\n4fc47e60-f1b5-469c-b666-e1f4570e749f-->|\"Supporting Supply Chain,\n[Implemented By]\"|0bf2547c-937c-41b6-814f-6284849271a1\n72a86eec-9734-4bc0-babb-4fec0aa7c9ff==>|\"publish results,\n[Solution Linking Wire]\"|0bf2547c-937c-41b6-814f-6284849271a1\nend\nsubgraph Implementation\n8947766b-da1c-4543-a09a-f917925fba80@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_date_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\n7767ccf0-0d4e-44ef-89ba-55c57f3ed54c@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000002.csv_date_column**\"}\n8947766b-da1c-4543-a09a-f917925fba80==>|\"copy [Data Mapping]\"|7767ccf0-0d4e-44ef-89ba-55c57f3ed54c\nb33f2d2b-ee61-4e35-b40a-5e4327f0b7de@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:29 BST 2025**\"}\n4f237bfb-0644-4eb6-b095-dd10eee1d803@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000004.csv_angleLeft_column**\"}\nb33f2d2b-ee61-4e35-b40a-5e4327f0b7de==>|\"copy [Data Mapping]\"|4f237bfb-0644-4eb6-b095-dd10eee1d803\n4ddf2266-e308-43b2-9412-7fef9967eb8b@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:17 BST 2025**\"}\na224e4dd-92b8-42db-a4f2-18b715ce36e7@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv_angleRight_column**\"}\n4ddf2266-e308-43b2-9412-7fef9967eb8b==>|\"copy [Data Mapping]\"|a224e4dd-92b8-42db-a4f2-18b715ce36e7\ned6736d7-835a-4f56-aa34-942e957bcb97@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_date_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\ne0c0f8eb-58c4-4b75-86d2-1fc09b95cf30@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000007.csv_date_column**\"}\ned6736d7-835a-4f56-aa34-942e957bcb97==>|\"copy [Data Mapping]\"|e0c0f8eb-58c4-4b75-86d2-1fc09b95cf30\nedf6f3e8-76ff-4652-b682-fe60d923a5db@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_patientId_column_archivedOn_Thu Apr 24 13:10:27 BST 2025**\"}\n9ded9d32-f6b7-4c8c-a3d3-a80b3de3ca8a@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv_patientId_column**\"}\nedf6f3e8-76ff-4652-b682-fe60d923a5db==>|\"copy [Data Mapping]\"|9ded9d32-f6b7-4c8c-a3d3-a80b3de3ca8a\n4b59ae22-f2bc-4a8c-ad3b-bf1abc0b3b4f@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_date_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\n328a8b5b-68c0-49be-ab7e-b80db53272c5@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000006.csv_date_column**\"}\n4b59ae22-f2bc-4a8c-ad3b-bf1abc0b3b4f==>|\"copy [Data Mapping]\"|328a8b5b-68c0-49be-ab7e-b80db53272c5\na7502835-1751-4873-b4c2-3717f9d666a6@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_patientId_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\n533b6986-3b75-4d83-8f23-e352addb282f@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000007.csv_patientId_column**\"}\na7502835-1751-4873-b4c2-3717f9d666a6==>|\"copy [Data Mapping]\"|533b6986-3b75-4d83-8f23-e352addb282f\nbbec4599-3198-4ade-8fb4-9bac5ffea9cd@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:29 BST 2025**\"}\nffedc1c0-6699-495c-8baa-bcee5796b7e3@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000004.csv_angleRight_column**\"}\nbbec4599-3198-4ade-8fb4-9bac5ffea9cd==>|\"copy [Data Mapping]\"|ffedc1c0-6699-495c-8baa-bcee5796b7e3\n99a4c486-f7e1-4c85-a97e-127abd2bffbd@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_patientId_column_archivedOn_Thu Apr 24 13:10:17 BST 2025**\"}\nc7307be1-2851-4407-ba83-2a524d4f63a1@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv_patientId_column**\"}\n99a4c486-f7e1-4c85-a97e-127abd2bffbd==>|\"copy [Data Mapping]\"|c7307be1-2851-4407-ba83-2a524d4f63a1\nfd14e14b-ad20-41b5-8750-3ccb3b983f72@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:27 BST 2025**\"}\n6af34df0-31fe-42db-9080-a6a9d5aebe4f@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv_angleLeft_column**\"}\nfd14e14b-ad20-41b5-8750-3ccb3b983f72==>|\"copy [Data Mapping]\"|6af34df0-31fe-42db-9080-a6a9d5aebe4f\n0bc93f0e-5ac1-43a3-a26b-596007359ae6@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_patientId_column_archivedOn_Thu Apr 24 13:10:31 BST 2025**\"}\n5dc432f5-869c-46f3-8bfc-6a9c1d89c04b@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000005.csv_patientId_column**\"}\n0bc93f0e-5ac1-43a3-a26b-596007359ae6==>|\"copy [Data Mapping]\"|5dc432f5-869c-46f3-8bfc-6a9c1d89c04b\n4af879a9-6aff-4541-b35a-95c222529865@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\nf1217ca5-d09f-4294-8749-743ae399573f@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000003.csv_angleLeft_column**\"}\n4af879a9-6aff-4541-b35a-95c222529865==>|\"copy [Data Mapping]\"|f1217ca5-d09f-4294-8749-743ae399573f\n5ae50d3e-f377-4327-b9c2-c47d2b0d9682@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_date_column_archivedOn_Thu Apr 24 13:10:33 BST 2025**\"}\n65be6ebc-50f3-4b27-ac6a-a97c2afd1f67@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000008.csv_date_column**\"}\n5ae50d3e-f377-4327-b9c2-c47d2b0d9682==>|\"copy [Data Mapping]\"|65be6ebc-50f3-4b27-ac6a-a97c2afd1f67\nad0e4161-45ef-4497-bb1a-a6b9b2b5e6a9@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:35 BST 2025**\"}\n2587d69a-c2ba-4c4d-a51d-fac3c7581301@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000009.csv_angleLeft_column**\"}\nad0e4161-45ef-4497-bb1a-a6b9b2b5e6a9==>|\"copy [Data Mapping]\"|2587d69a-c2ba-4c4d-a51d-fac3c7581301\n2db9b991-a803-4384-99db-229aee746e65@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_patientId_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\n1fb641b2-2cde-4b33-8b4a-2e2346ebf3dd@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000006.csv_patientId_column**\"}\n2db9b991-a803-4384-99db-229aee746e65==>|\"copy [Data Mapping]\"|1fb641b2-2cde-4b33-8b4a-2e2346ebf3dd\n9adbaed7-252c-4794-9ade-5ebfbf6b7073@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\nbc0948e6-2a3a-43a7-b190-5e6d14369349@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000007.csv_angleLeft_column**\"}\n9adbaed7-252c-4794-9ade-5ebfbf6b7073==>|\"copy [Data Mapping]\"|bc0948e6-2a3a-43a7-b190-5e6d14369349\n0ccb6d25-619d-4a8e-8eb2-8fb75ad9b3f6@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_patientId_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\nec916f60-90fc-4268-a46e-c4e3d24baf12@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000002.csv_patientId_column**\"}\n0ccb6d25-619d-4a8e-8eb2-8fb75ad9b3f6==>|\"copy [Data Mapping]\"|ec916f60-90fc-4268-a46e-c4e3d24baf12\nefe6d556-a8fb-425f-b501-e56c31fecdd3@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_date_column_archivedOn_Thu Apr 24 13:10:17 BST 2025**\"}\n65a7c9d8-8ba0-4ee5-bfe3-0c0abb8d038e@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv_date_column**\"}\nefe6d556-a8fb-425f-b501-e56c31fecdd3==>|\"copy [Data Mapping]\"|65a7c9d8-8ba0-4ee5-bfe3-0c0abb8d038e\n974f76f7-4ec2-47af-acab-738d8e524321@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\nd72991af-987b-4e31-a7d5-a37b2b26a68f@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000002.csv_angleLeft_column**\"}\n974f76f7-4ec2-47af-acab-738d8e524321==>|\"copy [Data Mapping]\"|d72991af-987b-4e31-a7d5-a37b2b26a68f\ne46dba10-a054-4456-b377-08961d4a011c@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_patientId_column_archivedOn_Thu Apr 24 13:10:29 BST 2025**\"}\n8a9d26df-27a3-4b8e-9832-b90f097d0206@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000004.csv_patientId_column**\"}\ne46dba10-a054-4456-b377-08961d4a011c==>|\"copy [Data Mapping]\"|8a9d26df-27a3-4b8e-9832-b90f097d0206\na2a1eeab-41fc-4f95-a0fd-3f4fbdf7323c@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:27 BST 2025**\"}\n38358833-7fdc-4374-9eef-bf14fa2d1d61@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv_angleRight_column**\"}\na2a1eeab-41fc-4f95-a0fd-3f4fbdf7323c==>|\"copy [Data Mapping]\"|38358833-7fdc-4374-9eef-bf14fa2d1d61\n5c52a7ac-4384-41dc-9256-0294d11a647a@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_patientId_column_archivedOn_Thu Apr 24 13:10:33 BST 2025**\"}\naa73d620-5620-4739-850c-cf6a42f3962b@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000008.csv_patientId_column**\"}\n5c52a7ac-4384-41dc-9256-0294d11a647a==>|\"copy [Data Mapping]\"|aa73d620-5620-4739-850c-cf6a42f3962b\n92c925a4-5770-48a5-bd7f-d42267443802@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_date_column_archivedOn_Thu Apr 24 13:10:31 BST 2025**\"}\n446fccdf-05d7-480b-833a-cc2513785259@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000005.csv_date_column**\"}\n92c925a4-5770-48a5-bd7f-d42267443802==>|\"copy [Data Mapping]\"|446fccdf-05d7-480b-833a-cc2513785259\n42b9c607-7b2e-4b98-b095-cccfb35b96bc@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:31 BST 2025**\"}\n7e1dc5cb-7f5d-423f-8d14-6c01db52099b@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000005.csv_angleLeft_column**\"}\n42b9c607-7b2e-4b98-b095-cccfb35b96bc==>|\"copy [Data Mapping]\"|7e1dc5cb-7f5d-423f-8d14-6c01db52099b\n88348764-a72e-4c9a-8edb-542eb157570e@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_date_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\n61f64fa7-185f-4a6b-828c-a1d81fc79a2d@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000003.csv_date_column**\"}\n88348764-a72e-4c9a-8edb-542eb157570e==>|\"copy [Data Mapping]\"|61f64fa7-185f-4a6b-828c-a1d81fc79a2d\n3363f617-29b1-4f6b-941e-58f3d01f8c3b@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\nc5acbb85-bc1c-4407-b487-f6a0f6a226fd@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000003.csv_angleRight_column**\"}\n3363f617-29b1-4f6b-941e-58f3d01f8c3b==>|\"copy [Data Mapping]\"|c5acbb85-bc1c-4407-b487-f6a0f6a226fd\n118e1057-49de-4365-886b-bce5a9ea0c8d@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\ne773d9aa-5784-46d9-b0bd-9a279c598bfa@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000006.csv_angleRight_column**\"}\n118e1057-49de-4365-886b-bce5a9ea0c8d==>|\"copy [Data Mapping]\"|e773d9aa-5784-46d9-b0bd-9a279c598bfa\n619d0f69-13b5-4eda-b3cc-cdb2f98ac074@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:35 BST 2025**\"}\nc1640aa4-5623-4bc9-85f1-14a112c28e25@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000009.csv_angleRight_column**\"}\n619d0f69-13b5-4eda-b3cc-cdb2f98ac074==>|\"copy [Data Mapping]\"|c1640aa4-5623-4bc9-85f1-14a112c28e25\n0a9dfd27-3a66-4b7d-8b56-cbd2277bae68@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\n2ec75f2c-8615-408b-ac0b-292df7203292@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000006.csv_angleLeft_column**\"}\n0a9dfd27-3a66-4b7d-8b56-cbd2277bae68==>|\"copy [Data Mapping]\"|2ec75f2c-8615-408b-ac0b-292df7203292\nbde7a350-9a9e-4fb4-a460-f9b5b6d08049@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\n91e8bda4-b867-430e-b671-87687287f847@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000002.csv_angleRight_column**\"}\nbde7a350-9a9e-4fb4-a460-f9b5b6d08049==>|\"copy [Data Mapping]\"|91e8bda4-b867-430e-b671-87687287f847\nabcdc5fa-ca22-492e-aefa-94c37646991b@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_date_column_archivedOn_Thu Apr 24 13:10:35 BST 2025**\"}\n9d35fe20-1f65-4e9c-8969-4e343a63c581@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000009.csv_date_column**\"}\nabcdc5fa-ca22-492e-aefa-94c37646991b==>|\"copy [Data Mapping]\"|9d35fe20-1f65-4e9c-8969-4e343a63c581\n2f6d91a0-06b0-41ed-81c0-ebfe16ce8584@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:33 BST 2025**\"}\n14b7792b-100d-4cb5-afe5-d5dcb773b6a7@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000008.csv_angleLeft_column**\"}\n2f6d91a0-06b0-41ed-81c0-ebfe16ce8584==>|\"copy [Data Mapping]\"|14b7792b-100d-4cb5-afe5-d5dcb773b6a7\n1b76d29b-5c7b-4a69-80b1-a86350cf01f7@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:33 BST 2025**\"}\ne5bf064c-90b0-42df-8ae4-e6b9fc95bdc3@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000008.csv_angleRight_column**\"}\n1b76d29b-5c7b-4a69-80b1-a86350cf01f7==>|\"copy [Data Mapping]\"|e5bf064c-90b0-42df-8ae4-e6b9fc95bdc3\n47741d18-c567-4783-bb48-9d4d56b3d306@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_date_column_archivedOn_Thu Apr 24 13:10:29 BST 2025**\"}\n32a5d4c1-b776-4d00-96d7-f6596358b3e9@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000004.csv_date_column**\"}\n47741d18-c567-4783-bb48-9d4d56b3d306==>|\"copy [Data Mapping]\"|32a5d4c1-b776-4d00-96d7-f6596358b3e9\n7f4527f8-0ab4-4a25-bdd6-93968bbd429c@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\n4b0d3ece-10a0-48d6-a8a1-232a589d1b7f@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000007.csv_angleRight_column**\"}\n7f4527f8-0ab4-4a25-bdd6-93968bbd429c==>|\"copy [Data Mapping]\"|4b0d3ece-10a0-48d6-a8a1-232a589d1b7f\n167206da-cdd8-425c-91b6-d94ac8b04133@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_patientId_column_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\n8131742c-a0d3-4d5f-a2e5-2be2775dfea4@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000003.csv_patientId_column**\"}\n167206da-cdd8-425c-91b6-d94ac8b04133==>|\"copy [Data Mapping]\"|8131742c-a0d3-4d5f-a2e5-2be2775dfea4\ndb5e4f32-84a5-41c4-959f-daad2be3bfd1@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_date_column_archivedOn_Thu Apr 24 13:10:27 BST 2025**\"}\nf7f90d02-4c4a-4846-ac5a-16f1807fb113@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv_date_column**\"}\ndb5e4f32-84a5-41c4-959f-daad2be3bfd1==>|\"copy [Data Mapping]\"|f7f90d02-4c4a-4846-ac5a-16f1807fb113\n790a4be9-1f89-4c6c-92ca-a3fed08c9d3c@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_angleLeft_column_archivedOn_Thu Apr 24 13:10:17 BST 2025**\"}\naa4f99e3-6526-429c-8480-338db9e15bc8@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv_angleLeft_column**\"}\n790a4be9-1f89-4c6c-92ca-a3fed08c9d3c==>|\"copy [Data Mapping]\"|aa4f99e3-6526-429c-8480-338db9e15bc8\na9141d33-5c48-4caa-b2ac-59500a751fec@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_patientId_column_archivedOn_Thu Apr 24 13:10:35 BST 2025**\"}\n7f4740fe-766a-4289-bcde-3fffb17f7940@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000009.csv_patientId_column**\"}\na9141d33-5c48-4caa-b2ac-59500a751fec==>|\"copy [Data Mapping]\"|7f4740fe-766a-4289-bcde-3fffb17f7940\n3545341b-0350-4083-a431-5d788cde1d23@{ shape: notch-pent, label: \"*Tabular Column*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_angleRight_column_archivedOn_Thu Apr 24 13:10:31 BST 2025**\"}\n69079bda-0a1e-40d4-b05a-836a530b1cd6@{ shape: rounded, label: \"*Tabular Column*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000005.csv_angleRight_column**\"}\n3545341b-0350-4083-a431-5d788cde1d23==>|\"copy [Data Mapping]\"|69079bda-0a1e-40d4-b05a-836a530b1cd6\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf@{ shape: rounded, label: \"*Data File Collection*\n**Data File Collection:Validated Incoming Weekly Measurements For Teddy Bear Drop Foot Clinical Trial**\"}\n4373f284-925c-4ea4-94cb-2a0e037ab32f@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000006.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|4373f284-925c-4ea4-94cb-2a0e037ab32f\n2d668829-1446-4cb1-951b-323d743d6eb7@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000003.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|2d668829-1446-4cb1-951b-323d743d6eb7\n7ef44d59-a988-49bc-a536-a89e58b9e13d@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000005.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|7ef44d59-a988-49bc-a536-a89e58b9e13d\nb9bb8b14-1557-4db5-8b7f-55016af532a9@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000008.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|b9bb8b14-1557-4db5-8b7f-55016af532a9\nbaa6081e-3ad4-44a8-89b7-606c27e44e79@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000009.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|baa6081e-3ad4-44a8-89b7-606c27e44e79\nd5074359-1804-4769-8db4-a480a77b0e5e@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000004.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|d5074359-1804-4769-8db4-a480a77b0e5e\n5474929e-ef14-46f8-bfc8-7c10a3fc1e20@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000000.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|5474929e-ef14-46f8-bfc8-7c10a3fc1e20\n20c0fff6-9477-448f-b144-7b6c1c72ed14@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000007.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|20c0fff6-9477-448f-b144-7b6c1c72ed14\n69e8370b-4b66-4e8c-9c45-21ef1a5f3289@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000002.csv**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Data Set Content\"|69e8370b-4b66-4e8c-9c45-21ef1a5f3289\nc930c48a-82d8-44a1-acfe-be07a244563a@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week4.csv_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\ne7e0d46a-6bc4-4c13-8ea6-42b477d56573@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService8ea6728a-2ab1-478d-ba26-699105ccc6b0**\"}\nc930c48a-82d8-44a1-acfe-be07a244563a==>|\"Data Flow\"|e7e0d46a-6bc4-4c13-8ea6-42b477d56573\n4e8f2545-d1d3-4bda-aca0-c8d63a42ec5e@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week1.csv_archivedOn_Thu Apr 24 13:10:27 BST 2025**\"}\n96a171e1-61f2-4453-bc95-d789e319122c@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService0f1bb121-2f82-4a38-9ff5-2788014566a8**\"}\n4e8f2545-d1d3-4bda-aca0-c8d63a42ec5e==>|\"Data Flow\"|96a171e1-61f2-4453-bc95-d789e319122c\n4d0cbb70-0c50-4556-8c30-961509c4429c@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService44a85bf1-be7e-471e-a5d3-ad2cb5596545**\"}\n4d0cbb70-0c50-4556-8c30-961509c4429c==>|\"Data Flow\"|b9bb8b14-1557-4db5-8b7f-55016af532a9\n1b52fbf3-e5d2-44d5-bd9e-f03001bd9073@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService289b636e-71ae-4320-b06a-7bf50ce5687a**\"}\n1b52fbf3-e5d2-44d5-bd9e-f03001bd9073==>|\"Data Flow\"|baa6081e-3ad4-44a8-89b7-606c27e44e79\nd0dd34ff-fa41-4ec3-8bdc-6712a7025e69@{ shape: rounded, label: \"*Deployed Software Component*\n**Apache Airflow DAG:populateDropfootSandbox**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"Copy Certified Data [Data Flow]\"|d0dd34ff-fa41-4ec3-8bdc-6712a7025e69\n48a7c4d4-659e-476e-9a74-55f8cab65a9e@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServiced8df7658-b8eb-4671-8fb2-1c3aaa4d058c**\"}\n48a7c4d4-659e-476e-9a74-55f8cab65a9e==>|\"Data Flow\"|69e8370b-4b66-4e8c-9c45-21ef1a5f3289\n822c55db-1bc6-445e-9d1f-911fee0d0f39@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week4.csv_archivedOn_Thu Apr 24 13:10:31 BST 2025**\"}\n7a853197-e8bb-4854-b737-f76d8dbdd6be@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServicefc54591f-4893-4eb3-b2de-77ddfe60655e**\"}\n822c55db-1bc6-445e-9d1f-911fee0d0f39==>|\"Data Flow\"|7a853197-e8bb-4854-b737-f76d8dbdd6be\ncd058d8a-b95d-40d7-8e3b-c5cedadf73cb@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week1.csv_archivedOn_Thu Apr 24 13:10:17 BST 2025**\"}\nfad6b7b0-93d8-4c26-8f93-6dcd9d1636a6@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService3db11b59-c7e8-4f20-a4ff-f141d192784b**\"}\ncd058d8a-b95d-40d7-8e3b-c5cedadf73cb==>|\"Data Flow\"|fad6b7b0-93d8-4c26-8f93-6dcd9d1636a6\ne7e0d46a-6bc4-4c13-8ea6-42b477d56573==>|\"Data Flow\"|20c0fff6-9477-448f-b144-7b6c1c72ed14\nc51c51ca-42d6-4dea-8291-8f7b33e66d45@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionServiceefeee970-11cd-4f49-bcbd-1b5bad17d7d6**\"}\nc51c51ca-42d6-4dea-8291-8f7b33e66d45==>|\"Data Flow\"|d5074359-1804-4769-8db4-a480a77b0e5e\n37a0d034-3fa9-42ca-8f14-190667d9b2ab@{ shape: rounded, label: \"*Deployed Database Schema*\n**PostgreSQL Relational Database Schema:LocalPostgreSQL1:clinical_trials.drop_foot**\"}\nd0dd34ff-fa41-4ec3-8bdc-6712a7025e69==>|\"Save Certified Data [Data Flow]\"|37a0d034-3fa9-42ca-8f14-190667d9b2ab\nc0f00fad-8068-47f1-8194-9bb6b50cc6fe@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week5.csv_archivedOn_Thu Apr 24 13:10:33 BST 2025**\"}\nc0f00fad-8068-47f1-8194-9bb6b50cc6fe==>|\"Data Flow\"|4d0cbb70-0c50-4556-8c30-961509c4429c\n86162bc8-9df0-4133-b6b8-8352c4bd34ce@{ shape: rounded, label: \"*Data Folder*\n**Data Folder:.landing-area/hospitals/oak-dene/clinical-trials/drop-foot**\"}\n1b98cdac-dd0a-4621-93db-99ef5a1098bc@{ shape: rounded, label: \"*Integration Connector*\n**Egeria:IntegrationGroup:Default:GeneralFilesMonitorIntegrationConnector**\"}\n86162bc8-9df0-4133-b6b8-8352c4bd34ce==>|\"new file notification [Data Flow]\"|1b98cdac-dd0a-4621-93db-99ef5a1098bc\nee8280cb-027b-4141-bb52-fa43172579f4@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService327dcf3f-402c-4c30-90dc-a491fffa1e6a**\"}\nee8280cb-027b-4141-bb52-fa43172579f4==>|\"Data Flow\"|2d668829-1446-4cb1-951b-323d743d6eb7\nf1b3def9-3821-4dcb-90da-f04d4d108fac@{ shape: rounded, label: \"*Governance Action Process*\n**Coco:GovernanceActionProcess:PROJ-CT-TBDF:Hampton Hospital:WeeklyMeasurements:Onboarding**\"}\n1b98cdac-dd0a-4621-93db-99ef5a1098bc==>|\"new file trigger [Data Flow]\"|f1b3def9-3821-4dcb-90da-f04d4d108fac\n7a853197-e8bb-4854-b737-f76d8dbdd6be==>|\"Data Flow\"|7ef44d59-a988-49bc-a536-a89e58b9e13d\n089c780d-2bb8-45f5-80f0-06fe0b2297ff@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week2.csv_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\n089c780d-2bb8-45f5-80f0-06fe0b2297ff==>|\"Data Flow\"|48a7c4d4-659e-476e-9a74-55f8cab65a9e\n48d5002a-37b4-438b-86b8-c113f1fde0cc@{ shape: rounded, label: \"*C S V File*\n**DataLake::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/coco-data-lake/research/clinical-trials/drop-foot/weekly-measurements/PROJ-CT-TBDF_000001.csv**\"}\n96a171e1-61f2-4453-bc95-d789e319122c==>|\"Data Flow\"|48d5002a-37b4-438b-86b8-c113f1fde0cc\n475c2839-2652-4839-bd23-6a5228cb7dbf@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week2.csv_archivedOn_Thu Apr 24 13:10:28 BST 2025**\"}\n475c2839-2652-4839-bd23-6a5228cb7dbf==>|\"Data Flow\"|ee8280cb-027b-4141-bb52-fa43172579f4\n0373a380-fbf4-4ea2-92f1-479a7676d134@{ shape: rounded, label: \"*Transient Embedded Process*\n**Egeria:MoveCopyDeleteFileGovernanceActionService917d70b1-251f-4e84-986c-eda425df696c**\"}\n0373a380-fbf4-4ea2-92f1-479a7676d134==>|\"Data Flow\"|4373f284-925c-4ea4-94cb-2a0e037ab32f\n1215334f-7da2-4bee-b515-a348998a4049@{ shape: rounded, label: \"*Organization*\n**Organization::Hampton Hospital**\"}\n00766073-de88-41c7-82e4-dc522d648d6c@{ shape: rounded, label: \"*Data Folder*\n**Data Folder:.landing-area/hospitals/hampton/clinical-trials/drop-foot**\"}\n1215334f-7da2-4bee-b515-a348998a4049==>|\"publish weekly measurements [Data Flow]\"|00766073-de88-41c7-82e4-dc522d648d6c\n6d7acf4e-0d61-4744-b7c3-e09de9269b70@{ shape: rounded, label: \"*Organization*\n**Organization::Oak Dene Hospital**\"}\n6d7acf4e-0d61-4744-b7c3-e09de9269b70==>|\"publish weekly measurements [Data Flow]\"|86162bc8-9df0-4133-b6b8-8352c4bd34ce\nfad6b7b0-93d8-4c26-8f93-6dcd9d1636a6==>|\"Data Flow\"|5474929e-ef14-46f8-bfc8-7c10a3fc1e20\n91678bb5-f5c6-47c0-bd86-50ac627c2a57@{ shape: rounded, label: \"*Governance Action Process*\n**Coco:GovernanceActionProcess:PROJ-CT-TBDF:Oak Dene Hospital:WeeklyMeasurements:Onboarding**\"}\n1b98cdac-dd0a-4621-93db-99ef5a1098bc==>|\"new file trigger [Data Flow]\"|91678bb5-f5c6-47c0-bd86-50ac627c2a57\ne1df5e1f-07d4-41cb-ba87-c0788236a3c3@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week5.csv_archivedOn_Thu Apr 24 13:10:35 BST 2025**\"}\ne1df5e1f-07d4-41cb-ba87-c0788236a3c3==>|\"Data Flow\"|1b52fbf3-e5d2-44d5-bd9e-f03001bd9073\n769fee63-4388-4f0f-b027-fabd18b22863@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Oak Dene Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week3.csv_archivedOn_Thu Apr 24 13:10:32 BST 2025**\"}\n769fee63-4388-4f0f-b027-fabd18b22863==>|\"Data Flow\"|0373a380-fbf4-4ea2-92f1-479a7676d134\n91678bb5-f5c6-47c0-bd86-50ac627c2a57==>|\"delivered validated file [Data Flow]\"|d2be0319-8bfe-42a3-899b-6e8c9658cbaf\n4ac4a4e3-63dd-438e-b845-c5583b1121ce@{ shape: notch-pent, label: \"*C S V File*\n**LandingArea::Hampton Hospital::PROJ-CT-TBDF::CSVFile::/Users/amandachessell/Code/ODPi/egeria-oak/egeria/open-metadata-distribution/omag-server-platform/build/unpacked/egeria-platform-5.3-SNAPSHOT-distribution.tar.gz/assembly/platform/landing-area/hospitals/hampton/clinical-trials/drop-foot/week3.csv_archivedOn_Thu Apr 24 13:10:30 BST 2025**\"}\n4ac4a4e3-63dd-438e-b845-c5583b1121ce==>|\"Data Flow\"|c51c51ca-42d6-4dea-8291-8f7b33e66d45\n00766073-de88-41c7-82e4-dc522d648d6c==>|\"new file notification [Data Flow]\"|1b98cdac-dd0a-4621-93db-99ef5a1098bc\nf1b3def9-3821-4dcb-90da-f04d4d108fac==>|\"delivered validated file [Data Flow]\"|d2be0319-8bfe-42a3-899b-6e8c9658cbaf\nd2b65243-6748-418d-86a0-5245b60b2191@{ shape: rounded, label: \"*Data Folder*\n**Unity Catalog Volume:http:/ /localhost:8087:clinical_trials.teddy_bear_drop_foot.weekly_measurements**\"}\nd2be0319-8bfe-42a3-899b-6e8c9658cbaf==>|\"get files from storage [Process Call]\"|d2b65243-6748-418d-86a0-5245b60b2191\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f-->|\"Implemented By\"|00766073-de88-41c7-82e4-dc522d648d6c\na5d4d638-6836-47e5-99d0-fdcde637e13f-->|\"Implemented By\"|d2be0319-8bfe-42a3-899b-6e8c9658cbaf\n1c150d6e-30cf-481c-9afb-3b06c9c9e78f-->|\"Implemented By\"|86162bc8-9df0-4133-b6b8-8352c4bd34ce\nd48f579f-76d3-4c49-b1b4-575f5645a9d0-->|\"Implemented By\"|37a0d034-3fa9-42ca-8f14-190667d9b2ab\n26c07ca4-3b8e-484b-812b-36c1ace4b275-->|\"Implemented By\"|d0dd34ff-fa41-4ec3-8bdc-6712a7025e69\n7f5dca65-50b4-4103-9ac7-3a406a09047a-->|\"Implemented By\"|91678bb5-f5c6-47c0-bd86-50ac627c2a57\n7f5dca65-50b4-4103-9ac7-3a406a09047a-->|\"Implemented By\"|f1b3def9-3821-4dcb-90da-f04d4d108fac\n07705e15-efff-4f80-8992-f04ac85e0ef1-->|\"Implemented By\"|1b98cdac-dd0a-4621-93db-99ef5a1098bc\nend\nend\nsubgraph Overview\na2caa20a-2cb1-4d35-a96a-6493bb9036d3@{ shape: flip-tri, label: \"*Information Supply Chain*\n**Clinical Trial Treatment Validation:PROJ-CT-TBDF**\"}\n14a43789-ae50-448e-b748-23c96bca79c1@{ shape: text, label: \"*Description*\n**Delivering data relating to the Teddy Bear Drop Foot Clinical Trial clinical trial from the hospitals to the Coco Researchers so that they can then determine the efficacy of the treatment to report to the regulators.**\"}\n7e65f803-6b1e-4d2c-85b8-ee69de7a1cde@{ shape: text, label: \"*Purpose*\n**Deliver patient measurement data from hospitals to data scientists in research.**\"}\n442b0057-af6c-469f-b80a-4c216d3b1450@{ shape: text, label: \"*Purpose*\n**Ensure incoming data is only from certified hospitals.**\"}\n8db7552e-f07c-4984-b476-83a373298394@{ shape: text, label: \"*Purpose*\n**Ensure incoming data is validated and catalogued.**\"}\na1fa06f0-bcca-4e13-b0de-1b178feb64e6@{ shape: text, label: \"*Purpose*\n**Ensure data and process owners are informed of key milestones and issues requiring attention.**\"}\nc96f31c9-297d-4b45-a842-eff9917e316b@{ shape: text, label: \"*Purpose*\n**Ensure the process of data capture and treatment validation is transparent and auditable.**\"}\n2d5b9653-8c15-4eea-a522-ce5ca7d07a6c@{ shape: text, label: \"*Purpose*\n**Ensure the treatment validation report is complete and regulatory compliant.**\"}\nend\nOverview~~~Details\n822c55db-1bc6-445e-9d1f-911fee0d0f39-. \"Anchors\" .->92c925a4-5770-48a5-bd7f-d42267443802\n822c55db-1bc6-445e-9d1f-911fee0d0f39-. \"Anchors\" .->0bc93f0e-5ac1-43a3-a26b-596007359ae6\n822c55db-1bc6-445e-9d1f-911fee0d0f39-. \"Anchors\" .->42b9c607-7b2e-4b98-b095-cccfb35b96bc\n822c55db-1bc6-445e-9d1f-911fee0d0f39-. \"Anchors\" .->3545341b-0350-4083-a431-5d788cde1d23\n4ac4a4e3-63dd-438e-b845-c5583b1121ce-. \"Anchors\" .->47741d18-c567-4783-bb48-9d4d56b3d306\n4ac4a4e3-63dd-438e-b845-c5583b1121ce-. \"Anchors\" .->b33f2d2b-ee61-4e35-b40a-5e4327f0b7de\n4ac4a4e3-63dd-438e-b845-c5583b1121ce-. \"Anchors\" .->e46dba10-a054-4456-b377-08961d4a011c\n4ac4a4e3-63dd-438e-b845-c5583b1121ce-. \"Anchors\" .->bbec4599-3198-4ade-8fb4-9bac5ffea9cd\n2d668829-1446-4cb1-951b-323d743d6eb7-. \"Anchors\" .->8131742c-a0d3-4d5f-a2e5-2be2775dfea4\n2d668829-1446-4cb1-951b-323d743d6eb7-. \"Anchors\" .->c5acbb85-bc1c-4407-b487-f6a0f6a226fd\n2d668829-1446-4cb1-951b-323d743d6eb7-. \"Anchors\" .->61f64fa7-185f-4a6b-828c-a1d81fc79a2d\n2d668829-1446-4cb1-951b-323d743d6eb7-. \"Anchors\" .->f1217ca5-d09f-4294-8749-743ae399573f\n475c2839-2652-4839-bd23-6a5228cb7dbf-. \"Anchors\" .->4af879a9-6aff-4541-b35a-95c222529865\n475c2839-2652-4839-bd23-6a5228cb7dbf-. \"Anchors\" .->167206da-cdd8-425c-91b6-d94ac8b04133\n475c2839-2652-4839-bd23-6a5228cb7dbf-. \"Anchors\" .->88348764-a72e-4c9a-8edb-542eb157570e\n475c2839-2652-4839-bd23-6a5228cb7dbf-. \"Anchors\" .->3363f617-29b1-4f6b-941e-58f3d01f8c3b\ne1df5e1f-07d4-41cb-ba87-c0788236a3c3-. \"Anchors\" .->ad0e4161-45ef-4497-bb1a-a6b9b2b5e6a9\ne1df5e1f-07d4-41cb-ba87-c0788236a3c3-. \"Anchors\" .->abcdc5fa-ca22-492e-aefa-94c37646991b\ne1df5e1f-07d4-41cb-ba87-c0788236a3c3-. \"Anchors\" .->a9141d33-5c48-4caa-b2ac-59500a751fec\ne1df5e1f-07d4-41cb-ba87-c0788236a3c3-. \"Anchors\" .->619d0f69-13b5-4eda-b3cc-cdb2f98ac074\nbaa6081e-3ad4-44a8-89b7-606c27e44e79-. \"Anchors\" .->9d35fe20-1f65-4e9c-8969-4e343a63c581\nbaa6081e-3ad4-44a8-89b7-606c27e44e79-. \"Anchors\" .->c1640aa4-5623-4bc9-85f1-14a112c28e25\nbaa6081e-3ad4-44a8-89b7-606c27e44e79-. \"Anchors\" .->7f4740fe-766a-4289-bcde-3fffb17f7940\nbaa6081e-3ad4-44a8-89b7-606c27e44e79-. \"Anchors\" .->2587d69a-c2ba-4c4d-a51d-fac3c7581301\n4373f284-925c-4ea4-94cb-2a0e037ab32f-. \"Anchors\" .->328a8b5b-68c0-49be-ab7e-b80db53272c5\n4373f284-925c-4ea4-94cb-2a0e037ab32f-. \"Anchors\" .->2ec75f2c-8615-408b-ac0b-292df7203292\n4373f284-925c-4ea4-94cb-2a0e037ab32f-. \"Anchors\" .->1fb641b2-2cde-4b33-8b4a-2e2346ebf3dd\n4373f284-925c-4ea4-94cb-2a0e037ab32f-. \"Anchors\" .->e773d9aa-5784-46d9-b0bd-9a279c598bfa\nb9bb8b14-1557-4db5-8b7f-55016af532a9-. \"Anchors\" .->65be6ebc-50f3-4b27-ac6a-a97c2afd1f67\nb9bb8b14-1557-4db5-8b7f-55016af532a9-. \"Anchors\" .->14b7792b-100d-4cb5-afe5-d5dcb773b6a7\nb9bb8b14-1557-4db5-8b7f-55016af532a9-. \"Anchors\" .->aa73d620-5620-4739-850c-cf6a42f3962b\nb9bb8b14-1557-4db5-8b7f-55016af532a9-. \"Anchors\" .->e5bf064c-90b0-42df-8ae4-e6b9fc95bdc3\n5474929e-ef14-46f8-bfc8-7c10a3fc1e20-. \"Anchors\" .->65a7c9d8-8ba0-4ee5-bfe3-0c0abb8d038e\n5474929e-ef14-46f8-bfc8-7c10a3fc1e20-. \"Anchors\" .->a224e4dd-92b8-42db-a4f2-18b715ce36e7\n5474929e-ef14-46f8-bfc8-7c10a3fc1e20-. \"Anchors\" .->aa4f99e3-6526-429c-8480-338db9e15bc8\n5474929e-ef14-46f8-bfc8-7c10a3fc1e20-. \"Anchors\" .->c7307be1-2851-4407-ba83-2a524d4f63a1\nc0f00fad-8068-47f1-8194-9bb6b50cc6fe-. \"Anchors\" .->1b76d29b-5c7b-4a69-80b1-a86350cf01f7\nc0f00fad-8068-47f1-8194-9bb6b50cc6fe-. \"Anchors\" .->5c52a7ac-4384-41dc-9256-0294d11a647a\nc0f00fad-8068-47f1-8194-9bb6b50cc6fe-. \"Anchors\" .->2f6d91a0-06b0-41ed-81c0-ebfe16ce8584\nc0f00fad-8068-47f1-8194-9bb6b50cc6fe-. \"Anchors\" .->5ae50d3e-f377-4327-b9c2-c47d2b0d9682\n4e8f2545-d1d3-4bda-aca0-c8d63a42ec5e-. \"Anchors\" .->edf6f3e8-76ff-4652-b682-fe60d923a5db\n4e8f2545-d1d3-4bda-aca0-c8d63a42ec5e-. \"Anchors\" .->fd14e14b-ad20-41b5-8750-3ccb3b983f72\n4e8f2545-d1d3-4bda-aca0-c8d63a42ec5e-. \"Anchors\" .->db5e4f32-84a5-41c4-959f-daad2be3bfd1\n4e8f2545-d1d3-4bda-aca0-c8d63a42ec5e-. \"Anchors\" .->a2a1eeab-41fc-4f95-a0fd-3f4fbdf7323c\n089c780d-2bb8-45f5-80f0-06fe0b2297ff-. \"Anchors\" .->0ccb6d25-619d-4a8e-8eb2-8fb75ad9b3f6\n089c780d-2bb8-45f5-80f0-06fe0b2297ff-. \"Anchors\" .->bde7a350-9a9e-4fb4-a460-f9b5b6d08049\n089c780d-2bb8-45f5-80f0-06fe0b2297ff-. \"Anchors\" .->974f76f7-4ec2-47af-acab-738d8e524321\n089c780d-2bb8-45f5-80f0-06fe0b2297ff-. \"Anchors\" .->8947766b-da1c-4543-a09a-f917925fba80\nd5074359-1804-4769-8db4-a480a77b0e5e-. \"Anchors\" .->ffedc1c0-6699-495c-8baa-bcee5796b7e3\nd5074359-1804-4769-8db4-a480a77b0e5e-. \"Anchors\" .->32a5d4c1-b776-4d00-96d7-f6596358b3e9\nd5074359-1804-4769-8db4-a480a77b0e5e-. \"Anchors\" .->8a9d26df-27a3-4b8e-9832-b90f097d0206\nd5074359-1804-4769-8db4-a480a77b0e5e-. \"Anchors\" .->4f237bfb-0644-4eb6-b095-dd10eee1d803\n769fee63-4388-4f0f-b027-fabd18b22863-. \"Anchors\" .->118e1057-49de-4365-886b-bce5a9ea0c8d\n769fee63-4388-4f0f-b027-fabd18b22863-. \"Anchors\" .->0a9dfd27-3a66-4b7d-8b56-cbd2277bae68\n769fee63-4388-4f0f-b027-fabd18b22863-. \"Anchors\" .->2db9b991-a803-4384-99db-229aee746e65\n769fee63-4388-4f0f-b027-fabd18b22863-. \"Anchors\" .->4b59ae22-f2bc-4a8c-ad3b-bf1abc0b3b4f\n7ef44d59-a988-49bc-a536-a89e58b9e13d-. \"Anchors\" .->69079bda-0a1e-40d4-b05a-836a530b1cd6\n7ef44d59-a988-49bc-a536-a89e58b9e13d-. \"Anchors\" .->446fccdf-05d7-480b-833a-cc2513785259\n7ef44d59-a988-49bc-a536-a89e58b9e13d-. \"Anchors\" .->5dc432f5-869c-46f3-8bfc-6a9c1d89c04b\n7ef44d59-a988-49bc-a536-a89e58b9e13d-. \"Anchors\" .->7e1dc5cb-7f5d-423f-8d14-6c01db52099b\nc930c48a-82d8-44a1-acfe-be07a244563a-. \"Anchors\" .->a7502835-1751-4873-b4c2-3717f9d666a6\nc930c48a-82d8-44a1-acfe-be07a244563a-. \"Anchors\" .->ed6736d7-835a-4f56-aa34-942e957bcb97\nc930c48a-82d8-44a1-acfe-be07a244563a-. \"Anchors\" .->7f4527f8-0ab4-4a25-bdd6-93968bbd429c\nc930c48a-82d8-44a1-acfe-be07a244563a-. \"Anchors\" .->9adbaed7-252c-4794-9ade-5ebfbf6b7073\n20c0fff6-9477-448f-b144-7b6c1c72ed14-. \"Anchors\" .->bc0948e6-2a3a-43a7-b190-5e6d14369349\n20c0fff6-9477-448f-b144-7b6c1c72ed14-. \"Anchors\" .->4b0d3ece-10a0-48d6-a8a1-232a589d1b7f\n20c0fff6-9477-448f-b144-7b6c1c72ed14-. \"Anchors\" .->e0c0f8eb-58c4-4b75-86d2-1fc09b95cf30\n20c0fff6-9477-448f-b144-7b6c1c72ed14-. \"Anchors\" .->533b6986-3b75-4d83-8f23-e352addb282f\n48d5002a-37b4-438b-86b8-c113f1fde0cc-. \"Anchors\" .->f7f90d02-4c4a-4846-ac5a-16f1807fb113\n48d5002a-37b4-438b-86b8-c113f1fde0cc-. \"Anchors\" .->38358833-7fdc-4374-9eef-bf14fa2d1d61\n48d5002a-37b4-438b-86b8-c113f1fde0cc-. \"Anchors\" .->9ded9d32-f6b7-4c8c-a3d3-a80b3de3ca8a\n48d5002a-37b4-438b-86b8-c113f1fde0cc-. \"Anchors\" .->6af34df0-31fe-42db-9080-a6a9d5aebe4f\ncd058d8a-b95d-40d7-8e3b-c5cedadf73cb-. \"Anchors\" .->4ddf2266-e308-43b2-9412-7fef9967eb8b\ncd058d8a-b95d-40d7-8e3b-c5cedadf73cb-. \"Anchors\" .->efe6d556-a8fb-425f-b501-e56c31fecdd3\ncd058d8a-b95d-40d7-8e3b-c5cedadf73cb-. \"Anchors\" .->790a4be9-1f89-4c6c-92ca-a3fed08c9d3c\ncd058d8a-b95d-40d7-8e3b-c5cedadf73cb-. \"Anchors\" .->99a4c486-f7e1-4c85-a97e-127abd2bffbd\n69e8370b-4b66-4e8c-9c45-21ef1a5f3289-. \"Anchors\" .->91e8bda4-b867-430e-b671-87687287f847\n69e8370b-4b66-4e8c-9c45-21ef1a5f3289-. \"Anchors\" .->d72991af-987b-4e31-a7d5-a37b2b26a68f\n69e8370b-4b66-4e8c-9c45-21ef1a5f3289-. \"Anchors\" .->7767ccf0-0d4e-44ef-89ba-55c57f3ed54c\n69e8370b-4b66-4e8c-9c45-21ef1a5f3289-. \"Anchors\" .->ec916f60-90fc-4268-a46e-c4e3d24baf12\nstyle 48bc201e-3d4e-4beb-bdb2-0fd9d134f6d5 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 47741d18-c567-4783-bb48-9d4d56b3d306 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle efe6d556-a8fb-425f-b501-e56c31fecdd3 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle d2b65243-6748-418d-86a0-5245b60b2191 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 42b9c607-7b2e-4b98-b095-cccfb35b96bc color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 99a4c486-f7e1-4c85-a97e-127abd2bffbd color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 4d0cbb70-0c50-4556-8c30-961509c4429c color:#000000, fill:#e0ab18, stroke:#004563\nstyle Segments color:#004563, fill:#FFFFFF, stroke:#004563\nstyle 8947766b-da1c-4543-a09a-f917925fba80 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle d72991af-987b-4e31-a7d5-a37b2b26a68f color:#000000, fill:#e0ab18, stroke:#004563\nstyle e0c0f8eb-58c4-4b75-86d2-1fc09b95cf30 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 6af34df0-31fe-42db-9080-a6a9d5aebe4f color:#000000, fill:#e0ab18, stroke:#004563\nstyle a5d4d638-6836-47e5-99d0-fdcde637e13f color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle Design color:#000000, fill:#b0c4de, stroke:#000000\nstyle 38635d38-f728-400d-a8ec-7c26e68b7c0f color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 0a9dfd27-3a66-4b7d-8b56-cbd2277bae68 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 7ef44d59-a988-49bc-a536-a89e58b9e13d color:#000000, fill:#e0ab18, stroke:#004563\nstyle 96a171e1-61f2-4453-bc95-d789e319122c color:#000000, fill:#e0ab18, stroke:#004563\nstyle ed6736d7-835a-4f56-aa34-942e957bcb97 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 20c0fff6-9477-448f-b144-7b6c1c72ed14 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 65a7c9d8-8ba0-4ee5-bfe3-0c0abb8d038e color:#000000, fill:#e0ab18, stroke:#004563\nstyle 2d5b9653-8c15-4eea-a522-ce5ca7d07a6c color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 2d668829-1446-4cb1-951b-323d743d6eb7 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 32a5d4c1-b776-4d00-96d7-f6596358b3e9 color:#000000, fill:#e0ab18, stroke:#004563\nstyle bbec4599-3198-4ade-8fb4-9bac5ffea9cd color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 7f4527f8-0ab4-4a25-bdd6-93968bbd429c color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 3363f617-29b1-4f6b-941e-58f3d01f8c3b color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 37a0d034-3fa9-42ca-8f14-190667d9b2ab color:#000000, fill:#e0ab18, stroke:#004563\nstyle Details color:#FFFFFF, fill:#FFFFFF, stroke:#FFFFFF\nstyle c930c48a-82d8-44a1-acfe-be07a244563a color:#004563, fill:#f8f8ff, stroke:#004563\nstyle bc0948e6-2a3a-43a7-b190-5e6d14369349 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 3545341b-0350-4083-a431-5d788cde1d23 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle a9141d33-5c48-4caa-b2ac-59500a751fec color:#004563, fill:#f8f8ff, stroke:#004563\nstyle c7307be1-2851-4407-ba83-2a524d4f63a1 color:#000000, fill:#e0ab18, stroke:#004563\nstyle ec916f60-90fc-4268-a46e-c4e3d24baf12 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 0ccb6d25-619d-4a8e-8eb2-8fb75ad9b3f6 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle f1b3def9-3821-4dcb-90da-f04d4d108fac color:#000000, fill:#e0ab18, stroke:#004563\nstyle 9d35fe20-1f65-4e9c-8969-4e343a63c581 color:#000000, fill:#e0ab18, stroke:#004563\nstyle b33f2d2b-ee61-4e35-b40a-5e4327f0b7de color:#004563, fill:#f8f8ff, stroke:#004563\nstyle e46dba10-a054-4456-b377-08961d4a011c color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 4373f284-925c-4ea4-94cb-2a0e037ab32f color:#000000, fill:#e0ab18, stroke:#004563\nstyle Implementation color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 4b59ae22-f2bc-4a8c-ad3b-bf1abc0b3b4f color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 2587d69a-c2ba-4c4d-a51d-fac3c7581301 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7edca02c-e726-4570-815c-280bdf5498b9 color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 0bf2547c-937c-41b6-814f-6284849271a1 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle db5e4f32-84a5-41c4-959f-daad2be3bfd1 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle fad6b7b0-93d8-4c26-8f93-6dcd9d1636a6 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 4ddf2266-e308-43b2-9412-7fef9967eb8b color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 1fb641b2-2cde-4b33-8b4a-2e2346ebf3dd color:#000000, fill:#e0ab18, stroke:#004563\nstyle 61f64fa7-185f-4a6b-828c-a1d81fc79a2d color:#000000, fill:#e0ab18, stroke:#004563\nstyle a224e4dd-92b8-42db-a4f2-18b715ce36e7 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 5dc432f5-869c-46f3-8bfc-6a9c1d89c04b color:#000000, fill:#e0ab18, stroke:#004563\nstyle 4e8f2545-d1d3-4bda-aca0-c8d63a42ec5e color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 442b0057-af6c-469f-b80a-4c216d3b1450 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 0bc93f0e-5ac1-43a3-a26b-596007359ae6 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 7a853197-e8bb-4854-b737-f76d8dbdd6be color:#000000, fill:#e0ab18, stroke:#004563\nstyle 769fee63-4388-4f0f-b027-fabd18b22863 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 48d5002a-37b4-438b-86b8-c113f1fde0cc color:#000000, fill:#e0ab18, stroke:#004563\nstyle 69079bda-0a1e-40d4-b05a-836a530b1cd6 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 619d0f69-13b5-4eda-b3cc-cdb2f98ac074 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 65be6ebc-50f3-4b27-ac6a-a97c2afd1f67 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 822c55db-1bc6-445e-9d1f-911fee0d0f39 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 2ec75f2c-8615-408b-ac0b-292df7203292 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 91e8bda4-b867-430e-b671-87687287f847 color:#000000, fill:#e0ab18, stroke:#004563\nstyle a7502835-1751-4873-b4c2-3717f9d666a6 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle e7e0d46a-6bc4-4c13-8ea6-42b477d56573 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 9ded9d32-f6b7-4c8c-a3d3-a80b3de3ca8a color:#000000, fill:#e0ab18, stroke:#004563\nstyle 5c52a7ac-4384-41dc-9256-0294d11a647a color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 8a9d26df-27a3-4b8e-9832-b90f097d0206 color:#000000, fill:#e0ab18, stroke:#004563\nstyle c0f00fad-8068-47f1-8194-9bb6b50cc6fe color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 8db7552e-f07c-4984-b476-83a373298394 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 533b6986-3b75-4d83-8f23-e352addb282f color:#000000, fill:#e0ab18, stroke:#004563\nstyle 328a8b5b-68c0-49be-ab7e-b80db53272c5 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 118e1057-49de-4365-886b-bce5a9ea0c8d color:#004563, fill:#f8f8ff, stroke:#004563\nstyle bde7a350-9a9e-4fb4-a460-f9b5b6d08049 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 00766073-de88-41c7-82e4-dc522d648d6c color:#000000, fill:#e0ab18, stroke:#004563\nstyle 790a4be9-1f89-4c6c-92ca-a3fed08c9d3c color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 4b0d3ece-10a0-48d6-a8a1-232a589d1b7f color:#000000, fill:#e0ab18, stroke:#004563\nstyle 0373a380-fbf4-4ea2-92f1-479a7676d134 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7767ccf0-0d4e-44ef-89ba-55c57f3ed54c color:#000000, fill:#e0ab18, stroke:#004563\nstyle cd058d8a-b95d-40d7-8e3b-c5cedadf73cb color:#004563, fill:#f8f8ff, stroke:#004563\nstyle ee8280cb-027b-4141-bb52-fa43172579f4 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 1c150d6e-30cf-481c-9afb-3b06c9c9e78f color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 07705e15-efff-4f80-8992-f04ac85e0ef1 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 4af879a9-6aff-4541-b35a-95c222529865 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 475c2839-2652-4839-bd23-6a5228cb7dbf color:#004563, fill:#f8f8ff, stroke:#004563\nstyle e1df5e1f-07d4-41cb-ba87-c0788236a3c3 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 38358833-7fdc-4374-9eef-bf14fa2d1d61 color:#000000, fill:#e0ab18, stroke:#004563\nstyle d2be0319-8bfe-42a3-899b-6e8c9658cbaf color:#000000, fill:#e0ab18, stroke:#004563\nstyle 4fc47e60-f1b5-469c-b666-e1f4570e749f color:#004563, fill:#b7c0c7, stroke:#004563\nstyle edf6f3e8-76ff-4652-b682-fe60d923a5db color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 9adbaed7-252c-4794-9ade-5ebfbf6b7073 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle aa4f99e3-6526-429c-8480-338db9e15bc8 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 6d7acf4e-0d61-4744-b7c3-e09de9269b70 color:#000000, fill:#e0ab18, stroke:#004563\nstyle ee2bb773-e630-4cf9-bdf1-7c2dd64fe4ec color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle ffedc1c0-6699-495c-8baa-bcee5796b7e3 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 14b7792b-100d-4cb5-afe5-d5dcb773b6a7 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 167206da-cdd8-425c-91b6-d94ac8b04133 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle f7f90d02-4c4a-4846-ac5a-16f1807fb113 color:#000000, fill:#e0ab18, stroke:#004563\nstyle d48f579f-76d3-4c49-b1b4-575f5645a9d0 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 2f6d91a0-06b0-41ed-81c0-ebfe16ce8584 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 48a7c4d4-659e-476e-9a74-55f8cab65a9e color:#000000, fill:#e0ab18, stroke:#004563\nstyle 04ae768e-3816-47bc-bddb-c9ae25018684 color:#004563, fill:#b7c0c7, stroke:#004563\nstyle 1b98cdac-dd0a-4621-93db-99ef5a1098bc color:#000000, fill:#e0ab18, stroke:#004563\nstyle fd14e14b-ad20-41b5-8750-3ccb3b983f72 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 5ae50d3e-f377-4327-b9c2-c47d2b0d9682 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 4ac4a4e3-63dd-438e-b845-c5583b1121ce color:#004563, fill:#f8f8ff, stroke:#004563\nstyle e4303326-e418-4f77-b8e7-fd5d34717594 color:#004563, fill:#b7c0c7, stroke:#004563\nstyle aa73d620-5620-4739-850c-cf6a42f3962b color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7e65f803-6b1e-4d2c-85b8-ee69de7a1cde color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle c96f31c9-297d-4b45-a842-eff9917e316b color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle c1640aa4-5623-4bc9-85f1-14a112c28e25 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 88348764-a72e-4c9a-8edb-542eb157570e color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 4f237bfb-0644-4eb6-b095-dd10eee1d803 color:#000000, fill:#e0ab18, stroke:#004563\nstyle a2a1eeab-41fc-4f95-a0fd-3f4fbdf7323c color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 92c925a4-5770-48a5-bd7f-d42267443802 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle d5074359-1804-4769-8db4-a480a77b0e5e color:#000000, fill:#e0ab18, stroke:#004563\nstyle 089c780d-2bb8-45f5-80f0-06fe0b2297ff color:#004563, fill:#f8f8ff, stroke:#004563\nstyle abcdc5fa-ca22-492e-aefa-94c37646991b color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 8131742c-a0d3-4d5f-a2e5-2be2775dfea4 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7f4740fe-766a-4289-bcde-3fffb17f7940 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 26c07ca4-3b8e-484b-812b-36c1ace4b275 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle e5bf064c-90b0-42df-8ae4-e6b9fc95bdc3 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 91678bb5-f5c6-47c0-bd86-50ac627c2a57 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 69e8370b-4b66-4e8c-9c45-21ef1a5f3289 color:#000000, fill:#e0ab18, stroke:#004563\nstyle ad0e4161-45ef-4497-bb1a-a6b9b2b5e6a9 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 1b76d29b-5c7b-4a69-80b1-a86350cf01f7 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle baa6081e-3ad4-44a8-89b7-606c27e44e79 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 1215334f-7da2-4bee-b515-a348998a4049 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 14a43789-ae50-448e-b748-23c96bca79c1 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle b9bb8b14-1557-4db5-8b7f-55016af532a9 color:#000000, fill:#e0ab18, stroke:#004563\nstyle Overview color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle 5474929e-ef14-46f8-bfc8-7c10a3fc1e20 color:#000000, fill:#e0ab18, stroke:#004563\nstyle f1217ca5-d09f-4294-8749-743ae399573f color:#000000, fill:#e0ab18, stroke:#004563\nstyle c51c51ca-42d6-4dea-8291-8f7b33e66d45 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7f5dca65-50b4-4103-9ac7-3a406a09047a color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle e773d9aa-5784-46d9-b0bd-9a279c598bfa color:#000000, fill:#e0ab18, stroke:#004563\nstyle 72a86eec-9734-4bc0-babb-4fec0aa7c9ff color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 1b52fbf3-e5d2-44d5-bd9e-f03001bd9073 color:#000000, fill:#e0ab18, stroke:#004563\nstyle b5c8da4c-f925-4cf1-8294-e43cd2c1a584 color:#FFFFFF, fill:#dda0dd, stroke:#000000\nstyle 974f76f7-4ec2-47af-acab-738d8e524321 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 86162bc8-9df0-4133-b6b8-8352c4bd34ce color:#000000, fill:#e0ab18, stroke:#004563\nstyle c5acbb85-bc1c-4407-b487-f6a0f6a226fd color:#000000, fill:#e0ab18, stroke:#004563\nstyle 2db9b991-a803-4384-99db-229aee746e65 color:#004563, fill:#f8f8ff, stroke:#004563\nstyle 446fccdf-05d7-480b-833a-cc2513785259 color:#000000, fill:#e0ab18, stroke:#004563\nstyle a2caa20a-2cb1-4d35-a96a-6493bb9036d3 color:#FFFFFF, fill:#004563, stroke:#b7c0c7\nstyle a1fa06f0-bcca-4e13-b0de-1b178feb64e6 color:#000000, fill:#F9F7ED, stroke:#b7c0c7\nstyle d0dd34ff-fa41-4ec3-8bdc-6712a7025e69 color:#000000, fill:#e0ab18, stroke:#004563\nstyle 7e1dc5cb-7f5d-423f-8d14-6c01db52099b color:#000000, fill:#e0ab18, stroke:#004563\n"

print(mermaid_graph)

render_mermaid(mermaid_graph)
